# 🏥 UbuntuCare Smarter Wait Times & Outcomes

## 📊 Overview
Synthetic dataset of ~250 000 patient visits simulating the overburdened South African public healthcare system with realistic patterns and systemic flaws.

## 📋 Feature Dictionary

| Feature Name | Description |
|--------------|-------------|
| **Patient_ID** | 👤 Unique patient identifier with some duplicates |
| **Province** | 🗺️ South African province where facility is located |
| **Facility_Name** | 🏢 Healthcare facility name with variations |
| **Age** | 🎂 Patient age in years with bimodal distribution |
| **Gender** | ⚧️ Patient gender identification |
| **Race_Demographic** | 🌍 Racial demographic classification |
| **Chief_Complaint** | 🤒 Primary reason for visit in free-text |
| **ICD-10_Code** | 📋 Medical classification code |
| **Triage_Category** | 🚨 Emergency severity assessment (SATS Colors) |
| **Department** | 🏥 Healthcare department visited |
| **Arrival_Date_Time** | ⏰ Date and time of patient arrival |
| **Day_of_Week** | 📅 Day of the week of visit |
| **Season** | 🌸 Seasonal period of visit |
| **Waiting_Time_Minutes** | ⏳ Time spent waiting for care (target variable) |
| **Outcome** | 📝 Final disposition of patient visit |
| **Insurance_Status** | 💰 Healthcare payment method |
| **Employment_Status** | 💼 Patient employment status |
| **Urban_Rural** | 🏙️ Geographic classification |
| **Facility_Referral_ID** | 🔄 Referral tracking identifier |
| **Follow_Up_Flag** | 📞 Follow-up requirement status |


## 🎲 Data Generation
Synthetically created using probabilistic rules mimicking real-world SA healthcare patterns with realistic relationships between features.

In [313]:
# IMPORT NECESSARY LIBRARIES
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy import stats
import statsmodels.api as sm
from datetime import datetime
import re
from fuzzywuzzy import process, fuzz
from typing import Union, List, Optional
import shutil


In [314]:
# ======================================
# PANDAS & SEABORN CONFIGURATION
# ======================================

# ----- PANDAS DISPLAY SETTINGS -----
# Display all columns and rows without truncation
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', 100)  # Limit rows to avoid excessive output
pd.set_option('display.width', 1000)    # Adjust display width for better readability
pd.set_option('display.colheader_justify', 'center')  # Center-align headers

# Floating-point precision (2 decimal places)
pd.options.display.float_format = '{:,.2f}'.format  # Adds thousand separators

# Improve performance with larger datasets
pd.set_option('compute.use_numexpr', True)  # Faster numerical operations
pd.set_option('mode.chained_assignment', 'warn')  # Warn on chained assignments



# Disable scientific notation for small numbers
pd.set_option('display.float_format', lambda x: '%.2f' % x)
np.set_printoptions(suppress=True)   # Suppress scientific notation in NumPy

# Suppress unnecessary warnings (optional)
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)  # Ignore FutureWarnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)  # Ignore PerformanceWarnings

In [315]:

# LOAD DATA
try:
    df =pd.read_csv(r'G:\Study\DATA SCINCE\PROJECTS\POTFOLIO\UbuntuCare Smarter Wait Times & Outcomes\06_Data\Raw\healthcare_data_output\enhanced_south_african_healthcare_dataset.csv')
    print(f"Shape: {df.shape} (Rows: {df.shape[0]}, Columns: {df.shape[1]})")
    print("Data loaded successfully.")

    # Display first few rows 
    display(df.head())


    # Basic info
    print("\n Data Overview:")
    display(df.info(verbose=True, show_counts=True))  # Shows non-null counts & dtypes

    #duplicate Rows
    print("\n Duplicate Rows:")
    display(df.duplicated().sum())

    #Missing Values
    print("\n Missing Values:")
    missing = df.isna().sum().to_frame(name='Missing Values')
    missing['% Missing'] = (missing['Missing Values'] / len(df)) * 100
    display(missing.sort_values("% Missing", ascending=False))


except FileNotFoundError:
    print("File not found. Please check the file path.")
except Exception as e:
    print(f"An error occurred: {e}")       

Shape: (250000, 29) (Rows: 250000, Columns: 29)
Data loaded successfully.


,Patient_ID,Province,Facility_Name,Age,Gender,Race_Demographic,Chief_Complaint,ICD-10_Code,Triage_Category,Department,Arrival_Date_Time,Day_of_Week,Season,Waiting_Time_Minutes,Outcome,Insurance_Status,Employment_Status,Urban_Rural,Follow_Up_Flag,SATS_Score,Respiratory_Rate,Heart_Rate,Oxygen_Saturation,Latitude,Longitude,Facility_Capacity,Staffing_Level,Facility_Referral_ID,Visit_Number
0,SAH-3839545,Eastern Cape,Nelson Mandela Academic Hospital,29.00,Male,Coloured,fractured arm,R69,Green,ANC,05/10/2022 08:15,Sun,Spring,306,Treated and Discharged,Gov Subsidy,Employed,Urban,No follow-up,10.00,12.00,0.00,98.00,-31.41,27.14,High,Normal,REF-33224,1
1,SAH-2498623,KwaZulu-Natal,Prince Mshiyeni Memorial Hospital,81.00,M,Black,Cough & fever,A15.0,Red,Paediatrics,20220828 1444,Wed,Winter,41,Admitted to Ward,Gov Subsidy,Retired,Urban,No follow-up,2.00,40.00,158.00,81.00,-28.84,29.05,Medium,Reduced,NaN,1
2,SAH-2152435,KwaZulu-Natal,Inkosi Albert Luthuli Central Hospital,3.00,F,Black,Cough & fever,J18.9,Red,Child Health,26/06/23 12.50 am,Thu,Winter,22,Admitted to Ward,Medical Aid,Student,Urban,No follow-up,1.00,40.00,123.00,77.00,-26.92,29.61,High,Reduced,REF-37630,1
3,SAH-4506463,Western Cape,Gugulethu CHC,5.00,M,Black,burns,NaN,Blue,OPD,30-Jan-2023 16h17,Thu,Summer,1194,Absconded,Gov Subsidy,Student,Rural,No follow-up,12.00,33.00,0.00,98.00,-35.02,16.65,Low,Limited,NaN,1
4,SAH-1601509,Eastern Cape,Frere Hospital,4.00,U,Black,Cough & fever,R05,Yellow,Paediatrics,30-Aug-2022 06h23,Friday,Winter,137,Admitted to Ward,Out-of-Pocket,Student,Urban,Follow-up scheduled,8.00,37.00,113.00,95.00,-33.98,24.41,Medium,Reduced,REF-25632,1



 Data Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 29 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Patient_ID            250000 non-null  object 
 1   Province              250000 non-null  object 
 2   Facility_Name         250000 non-null  object 
 3   Age                   245047 non-null  float64
 4   Gender                241341 non-null  object 
 5   Race_Demographic      242601 non-null  object 
 6   Chief_Complaint       250000 non-null  object 
 7   ICD-10_Code           232624 non-null  object 
 8   Triage_Category       242421 non-null  object 
 9   Department            250000 non-null  object 
 10  Arrival_Date_Time     242445 non-null  object 
 11  Day_of_Week           250000 non-null  object 
 12  Season                250000 non-null  object 
 13  Waiting_Time_Minutes  250000 non-null  int64  
 14  Outcome               250000 non-nu

None


 Duplicate Rows:


np.int64(0)


 Missing Values:


,Missing Values,% Missing
Facility_Referral_ID,174896,69.96
Respiratory_Rate,37645,15.06
Heart_Rate,37645,15.06
SATS_Score,37645,15.06
Oxygen_Saturation,37645,15.06
ICD-10_Code,17376,6.95
Gender,8659,3.46
Triage_Category,7579,3.03
Arrival_Date_Time,7555,3.02
Race_Demographic,7399,2.96


In [316]:
def descriptive_stats(dataframe):
    print("\n Descriptive Statistics Overview")

    # Numeric columns
    print("\n Numerical Columns:")
    display(dataframe.describe().T)  # Transposed for better readability

    # Categorical columns
    print("\n Categorical Columns:")
    display(dataframe.describe(include=['object', 'category']).T)  # Summary for categorical
descriptive_stats(df)



 Descriptive Statistics Overview

 Numerical Columns:


,count,mean,std,min,25%,50%,75%,max
Age,245047.00,27.81,62.01,-1.00,3.00,23.00,33.00,999.00
Waiting_Time_Minutes,250000.00,369.15,369.12,1.00,105.00,265.00,498.00,3750.00
SATS_Score,212355.00,8.75,2.79,1.00,7.00,10.00,11.00,12.00
Respiratory_Rate,212355.00,40.43,118.48,0.00,19.00,24.00,32.00,999.00
Heart_Rate,212355.00,119.02,113.19,0.00,88.00,107.00,125.00,999.00
Oxygen_Saturation,212355.00,96.06,4.81,70.00,95.00,98.00,99.00,100.00
Latitude,250000.00,-28.31,3.54,-35.72,-30.81,-27.41,-25.76,-20.90
Longitude,250000.00,27.30,4.14,16.62,26.81,28.29,29.78,32.90
Visit_Number,250000.00,1.06,0.25,1.00,1.00,1.00,1.00,4.00



 Categorical Columns:


,count,unique,top,freq
Patient_ID,250000,234450,SAH-5331196,4
Province,250000,5,Gauteng,87626
Facility_Name,250000,311,Addington Hospital,8564
Gender,241341,9,M,83936
Race_Demographic,242601,5,Black,181781
Chief_Complaint,250000,139,Cough & fever,45911
ICD-10_Code,232624,29,R69,108875
Triage_Category,242421,20,Green,92618
Department,250000,155,Paediatrics,34139
Arrival_Date_Time,242445,229982,2023-09-30,15


# 📊 Dataset Cleaning & Preprocessing Notes

## 1. Columns with Too Many Missing Values
- **Facility_Referral_ID**: >50% missing → will delete

## 2. Columns Not Needed for the Model
- **Patient_ID** → not necessary

---

## 3. Numeric Columns

| Column | Issue | Action |
|--------|-------|--------|
| **Arrival_Date_Time** | Object type, missing values | Convert to datetime and impute missing values using a smart method (e.g., forward fill, median time of day) |
| **Age** | Negative & extreme values (max=999), float | Replace invalid values with median, convert to integer |
| **Waiting_Time_Minutes** | Impossible extreme (e.g., 3750) | Replace extreme values with median |
| **Respiratory_Rate** | Unrealistic high values (e.g., 10000) | Cap values >40 bpm |
| **Heart_Rate** | Unrealistic high values (e.g., 999) | Replace with median or cap |
| **SATS_Score / Oxygen_Saturation** | Missing values | Impute with median |

---

## 4. Categorical Columns

| Feature | Issue(s) | Action Plan |
|---------|----------|-------------|
| **Facility_Name, Chief_Complaint, Department** | Typos, case sensitivity, spelling errors | 1. Standardize text: convert all entries to lowercase<br>2. Use fuzzy matching or manual mapping to consolidate duplicates and correct typos |
| **Race_Demographic, ICD-10_Code** | Missing data, typographical errors | 1. Standardize categories to fix typos<br>2. Impute missing values with the column mode |
| **Gender, Triage_Category** | Missing data, typographical errors | 1. Standardize categories<br>2. Impute missing values with mode |
| **Day_of_Week** | Inconsistent abbreviations (e.g., Mon, Monday) | Map all values to consistent full names or abbreviations |

---

## 💡 Summary / Manager Notes
- **Numeric columns:** Correct data types, replace extreme/unrealistic values with median  
- **Categorical columns:** Fix typos, standardize text, and impute missing values with mode  
- **Drop columns** with >50% missing values or irrelevant for the model  
- **Document every cleaning step** for reproducibility


# 📊 Dataset Cleaning & Preprocessing

### 1. Columns with Too Many Missing Values
- **Facility_Referral_ID**: >50% missing → will delete

### 2. Columns Not Needed for the Model
- **Patient_ID** → not necessary


In [ ]:
def clean_dataframe(df: pd.DataFrame, 
                   custom_remove_cols: Optional[List[str]] = None, 
                   missing_threshold: float = 0.5, 
                   remove_duplicates: bool = True,
                   low_variance_threshold: float = 0.0,
                   downcast_dtypes: bool = True,
                   verbose: bool = True) -> pd.DataFrame:
    """
    Advanced dataframe cleaning with multiple optimization techniques.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Input dataframe to clean
    custom_remove_cols : list, optional
        List of specific columns to remove
    missing_threshold : float, default 0.5
        Threshold for missing values percentage (0.0 to 1.0)
    remove_duplicates : bool, default True
        Whether to remove duplicate rows
    low_variance_threshold : float, default 0.0
        Remove columns with variance below this threshold (0.0 to 1.0)
    downcast_dtypes : bool, default True
        Optimize data types to reduce memory usage
    verbose : bool, default True
        Whether to print detailed cleaning summary
        
    Returns:
    --------
    pandas.DataFrame
        Cleaned and optimized dataframe
        
    Example:
    --------
    >>> df_cleaned = clean_dataframe(
    >>>     df, 
    >>>     custom_remove_cols=['Facility_Referral_ID', 'Patient_ID'],
    >>>     missing_threshold=0.5,
    >>>     remove_duplicates=True,
    >>>     low_variance_threshold=0.01,
    >>>     downcast_dtypes=True
    >>> )
    """
    # Create a copy to avoid modifying the original
    df_cleaned = df.copy()
    original_shape = df.shape
    original_memory = df.memory_usage(deep=True).sum() / 1024**2  # MB
    
    # Initialize columns to remove
    cols_to_remove = set()
    removal_reasons = {}
    
    # 1. Remove columns with more than missing_threshold% missing values 
    missing_percentage = df_cleaned.isnull().mean()
    high_missing_cols = missing_percentage[missing_percentage > missing_threshold].index.tolist()
    
    if high_missing_cols:
        cols_to_remove.update(high_missing_cols)
        removal_reasons.update({col: f"High missingness ({missing_percentage[col]:.1%})" 
                               for col in high_missing_cols})
    
    # 2. Remove custom specified columns if they exist 
    if custom_remove_cols:
        existing_cols = [col for col in custom_remove_cols if col in df_cleaned.columns]
        cols_to_remove.update(existing_cols)
        removal_reasons.update({col: "Manually specified" for col in existing_cols})
    
    # 3. Remove low variance columns (if threshold > 0) 
    if low_variance_threshold > 0:
        numeric_cols = df_cleaned.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            if col not in cols_to_remove:  # Skip already marked columns
                col_var = df_cleaned[col].var()
                col_range = df_cleaned[col].max() - df_cleaned[col].min()
                
                # Handle case where range is 0 (constant column)
                normalized_var = col_var / col_range if col_range > 0 else 0
                
                if normalized_var < low_variance_threshold:
                    cols_to_remove.add(col)
                    removal_reasons[col] = f"Low variance ({normalized_var:.6f})"
    
    # Remove the identified columns
    if cols_to_remove:
        df_cleaned.drop(columns=list(cols_to_remove), inplace=True, errors='ignore')
    
    # 4. Remove duplicate rows 
    duplicates_removed = 0
    if remove_duplicates:
        initial_rows = len(df_cleaned)
        df_cleaned.drop_duplicates(inplace=True)
        duplicates_removed = initial_rows - len(df_cleaned)
    
    # 5. Optimize data types to reduce memory usage 
    memory_saved = 0
    if downcast_dtypes:
        before_memory = df_cleaned.memory_usage(deep=True).sum() / 1024**2
        
        # Downcast numeric columns
        numeric_cols = df_cleaned.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            df_cleaned[col] = pd.to_numeric(df_cleaned[col], downcast='integer', errors='ignore')
            df_cleaned[col] = pd.to_numeric(df_cleaned[col], downcast='float', errors='ignore')
        
        # Downcast object columns to category if beneficial
        object_cols = df_cleaned.select_dtypes(include=['object']).columns
        for col in object_cols:
            if df_cleaned[col].nunique() / len(df_cleaned) < 0.5:  # If cardinality < 50%
                df_cleaned[col] = df_cleaned[col].astype('category')
        
        after_memory = df_cleaned.memory_usage(deep=True).sum() / 1024**2
        memory_saved = before_memory - after_memory
    
    # Print comprehensive summary 
    if verbose:
        print("🧹 DataFrame Cleaning Summary 🧹")
        print("=" * 50)
        print(f"Original shape: {original_shape} → Cleaned shape: {df_cleaned.shape}")
        print(f"Rows removed: {original_shape[0] - df_cleaned.shape[0]}")
        print(f"Columns removed: {len(cols_to_remove)}")
        print(f"Duplicate rows removed: {duplicates_removed}")
        print(f"Memory saved: {memory_saved:.2f} MB ({((original_memory - (original_memory - memory_saved)) / original_memory * 100):.1f}% reduction)")
        
        if cols_to_remove:
            print("\n Removed columns:")
            for col in cols_to_remove:
                print(f"  - {col}: {removal_reasons.get(col, 'No reason specified')}")
        
        # Show new data types
        print(f"\n Final data types:")
        print(df_cleaned.dtypes.value_counts())
    
    return df_cleaned

# Usage example
df_cleaned = clean_dataframe(
    df, 
    custom_remove_cols=['Facility_Referral_ID', 'Patient_ID'],
    missing_threshold=0.5,
    remove_duplicates=True,
    low_variance_threshold=0.01,  # Remove columns with very low variance
    downcast_dtypes=True,         # Optimize memory usage
    verbose=True                  # Show detailed summary
)

🧹 DataFrame Cleaning Summary 🧹
Original shape: (250000, 29) → Cleaned shape: (250000, 27)
Rows removed: 0
Columns removed: 2
Duplicate rows removed: 0
Memory saved: 238.87 MB (82.4% reduction)

 Removed columns:
  - Facility_Referral_ID: Manually specified
  - Patient_ID: Manually specified

 Final data types:
float32     7
category    1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
object      1
category    1
category    1
int16       1
category    1
category    1
category    1
category    1
category    1
category    1
category    1
int8        1
Name: count, dtype: int64


##  Numeric Columns

In [ ]:
# ---  AGE DATA CLEANING MODULE ---
def clean_age(series):
    """Clean unrealistic ages (<1, >120, or 0/negative) and return cleaned series. """
    series = series.copy()
    series = series.apply(lambda x: x if 1 <= x <= 120 else np.nan)
    return series

def impute_missing_age(series, strategy='median'):
    """Impute missing age values using median, mean, or mode. """
    if strategy == 'median':
        return series.fillna(series.median())
    elif strategy == 'mean':
        return series.fillna(series.mean())
    else:  # Default to median
        return series.fillna(series.median())

def create_age_categories(series):
    """Create standard age categories."""
    bins = [0, 18, 30, 40, 50, 60, 70, 80, 90, 120]
    labels = ['0-17 ','18-29 ','30-39 ','40-49 ','50-59 ','60-69 ','70-79 ','80-89 ','90+ ']
    return pd.cut(series, bins=bins, labels=labels, right=False)

# ---  Pipeline execution ---
print(" Starting age data cleaning...")
df_cleaned['Age'] = clean_age(df_cleaned['Age'])
print(f" Unrealistic ages cleaned! Missing values: {df_cleaned['Age'].isna().sum()}")

df_cleaned['Age'] = impute_missing_age(df_cleaned['Age'], strategy='median')
print(f" Missing ages imputed with median!")

df_cleaned['Age_Category'] = create_age_categories(df_cleaned['Age'])
print(f" Age categories created!")

# ---  Final Report ---
print("\n" + "="*50)
print(" Age cleaning completed successfully! ")
print(f" Final dataset shape: {df_cleaned.shape}")
print(f" New columns: {[col for col in df_cleaned.columns if 'Age' in col]}")
print(f" Age distribution:\n{df_cleaned['Age_Category'].value_counts().sort_index()}")
print("="*50)

 Starting age data cleaning...
 Unrealistic ages cleaned! Missing values: 23543
 Missing ages imputed with median!
 Age categories created!

 Age cleaning completed successfully! 🎉
 Final dataset shape: (250000, 28)
 New columns: ['Age', 'Age_Category']
 Age distribution:
Age_Category
0-17      80966
18-29     86807
30-39     41255
40-49      7969
50-59      8012
60-69      7971
70-79      8200
80-89      8031
90+         789
Name: count, dtype: int64


In [ ]:
def clean_waiting_time(series):
    """Cap extreme waiting times using IQR and fill missing values with median. """
    series = series.copy()
    
    # Calculate IQR boundaries
    Q1, Q3 = series.quantile([0.25, 0.75])
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
    median_val = series.median()
    
    print(f" IQR Analysis: Q1={Q1:.2f}, Q3={Q3:.2f}, IQR={IQR:.2f}")
    print(f" Boundaries: Lower={lower:.2f}, Upper={upper:.2f}")
    print(f" Median waiting time: {median_val:.2f} minutes")
    
    # Count outliers before capping
    outliers_count = series[(series < lower) | (series > upper)].count()
    print(f"  Detected {outliers_count} outliers ({outliers_count/series.count()*100:.1f}%)")
    
    # 🔧 Cap extreme values and fill missing
    series_cleaned = series.clip(lower=lower, upper=upper).fillna(median_val)
    
    # Count missing values filled
    missing_count = series.isna().sum()
    if missing_count > 0:
        print(f"🔍 Filled {missing_count} missing values with median")
    
    return series_cleaned

# ---  Execute cleaning ---
print("\n" + "="*60)
print(" Starting waiting time data cleaning... ⏱")
print("="*60)

df_cleaned['Waiting_Time_Minutes'] = clean_waiting_time(df_cleaned['Waiting_Time_Minutes'])

# ---  Final Report ---
print("\n" + "="*60)
print(" Waiting time cleaning completed successfully! ")
print(f" Final dataset shape: {df_cleaned.shape}")
print(f" Waiting time statistics after cleaning:")
print(f" Mean: {df_cleaned['Waiting_Time_Minutes'].mean():.2f} minutes")
print(f" Median: {df_cleaned['Waiting_Time_Minutes'].median():.2f} minutes")
print(f" Min: {df_cleaned['Waiting_Time_Minutes'].min():.2f} minutes")
print(f" Max: {df_cleaned['Waiting_Time_Minutes'].max():.2f} minutes")
print("="*60)


 Starting waiting time data cleaning... ⏱
 IQR Analysis: Q1=105.00, Q3=498.00, IQR=393.00
 Boundaries: Lower=-484.50, Upper=1087.50
 Median waiting time: 265.00 minutes
  Detected 13814 outliers (5.5%)

 Waiting time cleaning completed successfully! ⏱️✅
 Final dataset shape: (250000, 28)
 Waiting time statistics after cleaning:
 Mean: 347.93 minutes
 Median: 265.00 minutes
 Min: 1.00 minutes
 Max: 1087.50 minutes


In [ ]:
def clean_vital_signs(df):
    """
    Clean and impute vital signs data with capping of outliers and median imputation.
    Returns cleaned dataframe with processed vital signs. 
    
    Parameters:
    df (DataFrame): Input dataframe with vital signs columns
    
    Returns:
    DataFrame: Cleaned dataframe with processed vital signs
    """

    
    print(" Starting vital signs cleaning process...")
    print("="*55)
    
    # --- Respiratory_Rate  ---
    print("\n  Processing Respiratory Rate...")
    rr_upper_bound = 40
    rr_before = df_cleaned['Respiratory_Rate'].isna().sum()
    rr_outliers = (df_cleaned['Respiratory_Rate'] > rr_upper_bound).sum()
    
    df_cleaned['Respiratory_Rate'] = (
        df_cleaned['Respiratory_Rate']
        .clip(upper=rr_upper_bound)
        .fillna(df_cleaned['Respiratory_Rate'].median())
    )
    
    print(f"    Capped {rr_outliers} values > {rr_upper_bound} breaths/min")
    print(f"    Filled {rr_before} missing values with median")
    print(f"    New range: {df_cleaned['Respiratory_Rate'].min():.1f}-{df_cleaned['Respiratory_Rate'].max():.1f}")
    
    # --- Heart_Rate  ---
    print("\n Processing Heart Rate...")
    hr_upper_bound = 250
    hr_before = df_cleaned['Heart_Rate'].isna().sum()
    hr_outliers = (df_cleaned['Heart_Rate'] > hr_upper_bound).sum()
    
    valid_hr_median = df_cleaned.loc[df_cleaned['Heart_Rate'] <= hr_upper_bound, 'Heart_Rate'].median()
    
    df_cleaned['Heart_Rate'] = (
        df_cleaned['Heart_Rate']
        .where(df_cleaned['Heart_Rate'] <= hr_upper_bound, valid_hr_median)
        .fillna(valid_hr_median)
    )
    
    print(f"    Replaced {hr_outliers} values > {hr_upper_bound} bpm")
    print(f"    Filled {hr_before} missing values with median")
    print(f"    New range: {df_cleaned['Heart_Rate'].min():.1f}-{df_cleaned['Heart_Rate'].max():.1f}")
    
    # --- SATS_Score  ---
    print("\n Processing SATS Score...")
    sats_before = df_cleaned['SATS_Score'].isna().sum()
    sats_median = df_cleaned['SATS_Score'].median()
    
    df_cleaned['SATS_Score'] = df_cleaned['SATS_Score'].fillna(sats_median)
    
    print(f"    Filled {sats_before} missing values with median ({sats_median:.1f})")
    print(f"    Range: {df_cleaned['SATS_Score'].min():.1f}-{df_cleaned['SATS_Score'].max():.1f}")
    
    # --- Oxygen_Saturation 🫁 ---
    print("\n🫁 Processing Oxygen Saturation...")
    oxy_before = df_cleaned['Oxygen_Saturation'].isna().sum()
    oxy_median = df_cleaned['Oxygen_Saturation'].median()
    
    df_cleaned['Oxygen_Saturation'] = df_cleaned['Oxygen_Saturation'].fillna(oxy_median)
    
    print(f"    Filled {oxy_before} missing values with median ({oxy_median:.1f}%)")
    print(f"    Range: {df_cleaned['Oxygen_Saturation'].min():.1f}-{df_cleaned['Oxygen_Saturation'].max():.1f}%")
    
    return df_cleaned

# ---  Apply the cleaning function ---
print("\n" + "="*55)
print("Starting vital signs data cleaning...")
print("="*55)

df_cleaned = clean_vital_signs(df_cleaned)

# --- Final Summary ---
print("\n" + "="*55)
print(" Vital signs cleaning completed successfully! ")
print(f" Final dataset shape: {df_cleaned.shape}")
print("\n Vital Signs Summary:")
print(f"     Heart Rate: {df_cleaned['Heart_Rate'].mean():.1f} ± {df_cleaned['Heart_Rate'].std():.1f} bpm")
print(f"     Respiratory Rate: {df_cleaned['Respiratory_Rate'].mean():.1f} ± {df_cleaned['Respiratory_Rate'].std():.1f} breaths/min")
print(f"    SATS Score: {df_cleaned['SATS_Score'].mean():.1f} ± {df_cleaned['SATS_Score'].std():.1f}")
print(f"    O2 Saturation: {df_cleaned['Oxygen_Saturation'].mean():.1f} ± {df_cleaned['Oxygen_Saturation'].std():.1f}%")
print("="*55)


Starting vital signs data cleaning...
 Starting vital signs cleaning process...

  Processing Respiratory Rate...
    Capped 20797 values > 40 breaths/min
    Filled 37645 missing values with median
    New range: 0.0-40.0

 Processing Heart Rate...
    Replaced 3254 values > 250 bpm
    Filled 37645 missing values with median
    New range: 0.0-180.0

 Processing SATS Score...
    Filled 37645 missing values with median (10.0)
    Range: 1.0-12.0

🫁 Processing Oxygen Saturation...
    Filled 37645 missing values with median (98.0%)
    Range: 70.0-100.0%

 Vital signs cleaning completed successfully! 
 Final dataset shape: (250000, 28)

 Vital Signs Summary:
     Heart Rate: 105.4 ± 25.4 bpm
     Respiratory Rate: 25.1 ± 8.4 breaths/min
    SATS Score: 8.9 ± 2.6
    O2 Saturation: 96.4 ± 4.5%


##  Categorical Data Cleaning Module

In [321]:
# Expanded department mapping to handle your specific data
department_mapping = {
    # Antenatal
    'AN': 'Antenatal',
    'ANC': 'Antenatal',
    'ANCC': 'Antenatal',
    'ANTENATAL': 'Antenatal',
    'Antanatal': 'Antenatal',
    'Antenata': 'Antenatal',
    'Antenatal': 'Antenatal',
    'Antenatall': 'Antenatal',
    'antenatal': 'Antenatal',
    'anc': 'Antenatal',

    # Emergency
    'A&E': 'Emergency',
    'A&': 'Emergency',
    'A&EE': 'Emergency',
    'EMERGENCY': 'Emergency',
    'ER': 'Emergency',
    'Emargancy': 'Emergency',
    'Emergenc': 'Emergency',
    'Emergency': 'Emergency',
    'Emergency Dept': 'Emergency',
    'Emergencyy': 'Emergency',
    'Emergnc': 'Emergency',
    'Emergncy': 'Emergency',
    'emergency': 'Emergency',
    'emergncy': 'Emergency',
    'Emergncyy': 'Emergency',
    'EMERGNNCY': 'Emergency',
    'emergency dept': 'Emergency',
    'Emergency Dep': 'Emergency',
    'Emergency Deptt': 'Emergency',
    'EMERGENCY DEPT': 'Emergency',
    'er': 'Emergency',
    'ACCIDENT': 'Emergency',
    'Accident': 'Emergency',
    'accident': 'Emergency',
    'a&e': 'Emergency',  # Added
    'EMERGNCY': 'Emergency',  # Added
    'Emargancy Dapt': 'Emergency',  # Added
    'Emargncy': 'Emergency',  # Added

    # HIV/TB Clinic
    'HIV Clini': 'HIV/TB Clinic',
    'HIV Clinic': 'HIV/TB Clinic',
    'HIV Clinicc': 'HIV/TB Clinic',
    'HIV/TB CLINIC': 'HIV/TB Clinic',
    'HIV/TB Clini': 'HIV/TB Clinic',
    'HIV/TB Clinic': 'HIV/TB Clinic',
    'HIV/TB Clinicc': 'HIV/TB Clinic',
    'hiv/tb clinic': 'HIV/TB Clinic',
    'HIV CLINIC': 'HIV/TB Clinic',
    'hiv clinic': 'HIV/TB Clinic',
    'arv clinic': 'HIV/TB Clinic',
    'ARV Clinic': 'HIV/TB Clinic',
    'ARV Clinicc': 'HIV/TB Clinic',
    'ARV Clini': 'HIV/TB Clinic',
    'ARV CLINIC': 'HIV/TB Clinic',

    # Maternity
    'Maternit': 'Maternity',
    'Maternity': 'Maternity',
    'Maternityy': 'Maternity',
    'MATERNITY': 'Maternity',
    'maternity': 'Maternity',
    'Matarnity': 'Maternity',
    'MAT': 'Maternity',
    'Mat': 'Maternity',

    # Medication Collection
    'Madication Collaction': 'Medication Collection',
    'Medication Collectio': 'Medication Collection',
    'Medication Collection': 'Medication Collection',
    'Medication Collectionn': 'Medication Collection',
    'MEDICATION COLLECTION': 'Medication Collection',
    'medication collection': 'Medication Collection',
    'MED COLL': 'Medication Collection',
    'Med Coll': 'Medication Collection',

    # OPD / Outpatient
    'OP': 'OPD',
    'OPD': 'OPD',
    'OPDD': 'OPD',
    'Obstatrics': 'OPD',
    'Obstetrics': 'OPD',
    'Obstetricss': 'OPD',
    'Outpatiant': 'OPD',
    'Outpatiant Dapt': 'OPD',
    'Outpatien': 'OPD',
    'Outpatient': 'OPD',
    'Outpatient Dep': 'OPD',
    'Outpatient Dept': 'OPD',
    'Outpatient Deptt': 'OPD',
    'Outpatientt': 'OPD',
    'opd': 'OPD',
    'outpatient': 'OPD',
    'outpatient dept': 'OPD',
    'OUTPATIENT DEPT': 'OPD',
    'Obstetric': 'OPD',
    'OBSTETRICS': 'OPD',
    'obstetrics': 'OPD',
    'CLINIC': 'OPD',
    'clinic': 'OPD',
    'OUTPATIENT': 'OPD',  # Added

    # Paediatrics / Pediatrics / Child Health
    'PAEDIATRICS': 'Paediatrics',
    'Paadiatrics': 'Paediatrics',
    'Paads': 'Paediatrics',
    'Padiatrics': 'Paediatrics',
    'Paed': 'Paediatrics',
    'Paediatric': 'Paediatrics',
    'Paediatrics': 'Paediatrics',
    'Paediatricss': 'Paediatrics',
    'Paeds': 'Paediatrics',
    'Paedss': 'Paediatrics',
    'Pediatric': 'Paediatrics',
    'Pediatrics': 'Paediatrics',
    'Pediatricss': 'Paediatrics',
    'paediatrics': 'Paediatrics',
    'paeds': 'Paediatrics',
    'PEDS': 'Paediatrics',
    'Peds': 'Paediatrics',
    'PEDIATRICS': 'Paediatrics',  # Added
    'pediatrics': 'Paediatrics',  # Added
    'PAEDS': 'Paediatrics',  # Added
    # Child Health variations
    'CHILD HEALTH': 'Paediatrics',
    'Child Haalth': 'Paediatrics',
    'Child Healt': 'Paediatrics',
    'Child Health': 'Paediatrics',
    'Child Healthh': 'Paediatrics',
    'child health': 'Paediatrics',

    # Pharmacy / Dispensary
    'PHARMACY': 'Pharmacy',
    'Pharmac': 'Pharmacy',
    'Pharmacy': 'Pharmacy',
    'Pharmacyy': 'Pharmacy',
    'pharmacy': 'Pharmacy',
    'PHARM': 'Pharmacy',
    'Pharm': 'Pharmacy',
    # Dispensary variations
    'DISPENSARY': 'Pharmacy',
    'Dispansary': 'Pharmacy',
    'Dispensar': 'Pharmacy',
    'Dispensary': 'Pharmacy',
    'Dispensaryy': 'Pharmacy',
    'dispensary': 'Pharmacy',

    # Surgery / Surgical
    'Surgary': 'Surgery',
    'Surger': 'Surgery',
    'Surgery': 'Surgery',
    'Surgeryy': 'Surgery',
    'Surgical': 'Surgery',
    'Surgical Ward': 'Surgery',
    'Surgical Wardd': 'Surgery',
    'Surgical War': 'Surgery',
    'Surgicall': 'Surgery',
    'Surgica': 'Surgery',
    'surgery': 'Surgery',
    'SURGERY': 'Surgery',
    'surgical': 'Surgery',
    'SURGICAL': 'Surgery',
    'surgical ward': 'Surgery',
    'SURGICAL WARD': 'Surgery',
    'SURG': 'Surgery',
    'Surg': 'Surgery',

    # TB Clinic
    'TB Clini': 'TB Clinic',
    'TB Clinic': 'TB Clinic',
    'TB Clinicc': 'TB Clinic',
    'tb clinic': 'TB Clinic',
    'TB CLINIC': 'TB Clinic',
    'TB': 'TB Clinic',

    # Trauma
    'TRAUMA': 'Trauma',
    'Traum': 'Trauma',
    'Trauma': 'Trauma',
    'Trauma Cantra': 'Trauma',
    'Trauma Centr': 'Trauma',
    'Trauma Centre': 'Trauma',
    'Trauma Centree': 'Trauma',
    'Trauma Uni': 'Trauma',
    'Trauma Unit': 'Trauma',
    'Trauma Unitt': 'Trauma',
    'Traumaa': 'Trauma',
    'trauma': 'Trauma',
    'TRAUMA CENTRE': 'Trauma',
    'trauma centre': 'Trauma',
    'TRAUMA UNIT': 'Trauma',
    'trauma unit': 'Trauma'
}

# Enhanced function for better matching
def standardize_department_name_enhanced(input_name):
    """Convert any department variation to its canonical name with enhanced matching"""
    if not input_name or not isinstance(input_name, str):
        return input_name
    
    cleaned_input = input_name.strip()
    
    # Try exact match first
    if cleaned_input in department_mapping:
        return department_mapping[cleaned_input]
    
    # Try case-insensitive match
    lower_input = cleaned_input.lower()
    for key, value in department_mapping.items():
        if key.lower() == lower_input:
            return value
    
    # Try partial matching for common patterns
    if 'child' in lower_input or 'paed' in lower_input or 'ped' in lower_input:
        return 'Paediatrics'
    if 'dispens' in lower_input:
        return 'Pharmacy'
    if 'emerg' in lower_input or 'a&' in lower_input:
        return 'Emergency'
    if 'outpatient' in lower_input or 'opd' in lower_input:
        return 'OPD'
    
    return cleaned_input  # Return original if no good match found

# Apply the enhanced standardization
df_cleaned['Department'] = df_cleaned['Department'].apply(standardize_department_name_enhanced)

# Re-analyze the results
print("ANALYZING YOUR DATA AFTER ENHANCED CLEANING:")
analyze_actual_data(df_cleaned)

# Show value counts of the cleaned departments
print("\nCLEANED DEPARTMENT VALUE COUNTS:")
print(df_cleaned['Department'].value_counts().head(20))  # Show top 20 for brevity

ANALYZING YOUR DATA AFTER ENHANCED CLEANING:
UNIQUE DEPARTMENT NAMES - BEFORE STANDARDIZATION
 1. Antenatal
 2. Emergency
 3. HIV/TB Clinic
 4. Maternity
 5. Medication Collection
 6. OPD
 7. Paediatrics
 8. Pharmacy
 9. Surgery
10. TB Clinic
11. Trauma

UNIQUE DEPARTMENT NAMES - AFTER STANDARDIZATION
 1. Antenatal
 2. Emergency
 3. HIV/TB Clinic
 4. Maternity
 5. Medication Collection
 6. OPD
 7. Paediatrics
 8. Pharmacy
 9. Surgery
10. TB Clinic
11. Trauma

STATISTICS
Total records: 250000
Unique departments before: 11
Unique departments after: 11
Reduction in unique values: 0 (0.0%)

SAMPLE MAPPING EXAMPLES
'OPD' → (no mapping found)
'HIV/TB Clinic' → (no mapping found)
'Antenatal' → (no mapping found)
'Paediatrics' → (no mapping found)
'Trauma' → (no mapping found)
'Pharmacy' → (no mapping found)
'Emergency' → (no mapping found)
'Medication Collection' → (no mapping found)
'TB Clinic' → (no mapping found)
'Surgery' → (no mapping found)

CLEANED DEPARTMENT VALUE COUNTS:
Department
P

In [322]:
# Expanded facility mapping to handle all the variations in your data
facility_map = {
    # Addington Hospital variations
    "ADDINGTON HOSPITAL": "Addington Hospital",
    "Addington Hospita": "Addington Hospital",
    "Addington Hospital": "Addington Hospital",
    "Addington Hospitall": "Addington Hospital",
    "addington hospital": "Addington Hospital",
    
    # Baragwanath Hospital variations
    "BARAGWANATH HOSPITAL": "Chris Hani Baragwanath Hospital",
    "Baragwanath Hospita": "Chris Hani Baragwanath Hospital",
    "Baragwanath Hospital": "Chris Hani Baragwanath Hospital",
    "Baragwanath Hospitall": "Chris Hani Baragwanath Hospital",
    "baragwanath hospital": "Chris Hani Baragwanath Hospital",
    "CHRIS HANI BARA": "Chris Hani Baragwanath Hospital",
    "CHRIS HANI BARAGWANATH ACAD (SOWETO)": "Chris Hani Baragwanath Hospital",
    "Chris Hani Bar": "Chris Hani Baragwanath Hospital",
    "Chris Hani Bara": "Chris Hani Baragwanath Hospital",
    "Chris Hani Baraa": "Chris Hani Baragwanath Hospital",
    "Chris Hani Baragwanath ACAD (Sowato)": "Chris Hani Baragwanath Hospital",
    "Chris Hani Baragwanath ACAD (Soweto": "Chris Hani Baragwanath Hospital",
    "Chris Hani Baragwanath ACAD (Soweto)": "Chris Hani Baragwanath Hospital",
    "Chris Hani Baragwanath ACAD (Soweto))": "Chris Hani Baragwanath Hospital",
    "chris hani bara": "Chris Hani Baragwanath Hospital",
    "chris hani baragwanath acad (soweto)": "Chris Hani Baragwanath Hospital",
    
    # Charlotte Maxeke Hospital variations
    "CHARLOTTE MAXEKE HOSPITAL": "Charlotte Maxeke Hospital",
    "Charlotte Maxeke Hospita": "Charlotte Maxeke Hospital",
    "Charlotte Maxeke Hospital": "Charlotte Maxeke Hospital",
    "Charlotte Maxeke Hospitall": "Charlotte Maxeke Hospital",
    "Charlotte Maxeke Johannesburg Academic Hospital": "Charlotte Maxeke Hospital",
    "charlotte maxeke hospital": "Charlotte Maxeke Hospital",
    
    # CHB Hospital variations
    "CHB HOSPITAL": "CHB Hospital",
    "CHB Hospita": "CHB Hospital",
    "CHB Hospital": "CHB Hospital",
    "CHB Hospitall": "CHB Hospital",
    "chb hospital": "CHB Hospital",
    
    # Diepkloof Clinic variations
    "DIEPKLOOF CLINIC": "Diepkloof Clinic",
    "DIEPKLOOF CLININC": "Diepkloof Clinic",
    "Diepkloof Clini": "Diepkloof Clinic",
    "Diepkloof Clinic": "Diepkloof Clinic",
    "Diepkloof Clinicc": "Diepkloof Clinic",
    "Diepkloof Clinin": "Diepkloof Clinic",
    "Diepkloof Clininc": "Diepkloof Clinic",
    "Diepkloof Clinincc": "Diepkloof Clinic",
    "diepkloof clinic": "Diepkloof Clinic",
    "diepkloof clininc": "Diepkloof Clinic",
    
    # Dora Nginza Hospital variations
    "DORA NGINZA HOSPITAL": "Dora Nginza Hospital",
    "Dora Nginza Hospita": "Dora Nginza Hospital",
    "Dora Nginza Hospital": "Dora Nginza Hospital",
    "Dora Nginza Hospitall": "Dora Nginza Hospital",
    "dora nginza hospital": "Dora Nginza Hospital",
    
    # Frere Hospital variations
    "FRERE HOSPITAL": "Frere Hospital",
    "Frere Hospita": "Frere Hospital",
    "Frere Hospital": "Frere Hospital",
    "Frere Hospitall": "Frere Hospital",
    "frere hospital": "Frere Hospital",
    
    # Greys Hospital variations
    "GREYS HOSPITAL": "Greys Hospital",
    "Greys Hospita": "Greys Hospital",
    "Greys Hospital": "Greys Hospital",
    "Greys Hospitall": "Greys Hospital",
    "greys hospital": "Greys Hospital",
    
    # Groote Schuur Hospital variations
    "GROOTE SCHUUR": "Groote Schuur Hospital",
    "GROOTE SCHUUR HOSPITAL (CPT)": "Groote Schuur Hospital",
    "GSH CP": "Groote Schuur Hospital",
    "GSH CPT": "Groote Schuur Hospital",
    "GSH CPTT": "Groote Schuur Hospital",
    "Groota Schuur Hospital (CPT)": "Groote Schuur Hospital",
    "Groote Schuur": "Groote Schuur Hospital",
    "Groote Schuur Hospital": "Groote Schuur Hospital",
    "Groote Schuur Hospital (CPT": "Groote Schuur Hospital",
    "Groote Schuur Hospital (CPT)": "Groote Schuur Hospital",
    "Groote Schuur Hospital (CPT))": "Groote Schuur Hospital",
    "Groote Schuurr": "Groote Schuur Hospital",
    "groote schuur": "Groote Schuur Hospital",
    "groote schuur hospital (cpt)": "Groote Schuur Hospital",
    "gsh cpt": "Groote Schuur Hospital",
    
    # Gugulethu CHC variations
    "GUGS CHC": "Gugulethu CHC",
    "GUGULETHU CHC": "Gugulethu CHC",
    "GUGULETHU CLINIC": "Gugulethu CHC",
    "Gugs CH": "Gugulethu CHC",
    "Gugs CHC": "Gugulethu CHC",
    "Gugs CHCC": "Gugulethu CHC",
    "Gugulathu CHC": "Gugulethu CHC",
    "Gugulethu CH": "Gugulethu CHC",
    "Gugulethu CHC": "Gugulethu CHC",
    "Gugulethu CHCC": "Gugulethu CHC",
    "Gugulethu Clini": "Gugulethu CHC",
    "Gugulethu Clinic": "Gugulethu CHC",
    "Gugulethu Clinicc": "Gugulethu CHC",
    "Gugulethu Community Health Centre": "Gugulethu CHC",
    "gugs chc": "Gugulethu CHC",
    "gugulethu chc": "Gugulethu CHC",
    "gugulethu clinic": "Gugulethu CHC",
    
    # Helen Joseph Hospital variations
    "HELEN JOSEPH HOSPITAL": "Helen Joseph Hospital",
    "Helen Joseph Hospita": "Helen Joseph Hospital",
    "Helen Joseph Hospital": "Helen Joseph Hospital",
    "Helen Joseph Hospitall": "Helen Joseph Hospital",
    "helen joseph hospital": "Helen Joseph Hospital",
    
    # Inkosi Albert Luthuli Central Hospital variations
    "INKOSI ALBERT LUTHULI CENTRAL HOSPITAL": "Inkosi Albert Luthuli Central Hospital",
    "Inkosi Albert Luthuli Central Hospita": "Inkosi Albert Luthuli Central Hospital",
    "Inkosi Albert Luthuli Central Hospital": "Inkosi Albert Luthuli Central Hospital",
    "Inkosi Albert Luthuli Central Hospitall": "Inkosi Albert Luthuli Central Hospital",
    "inkosi albert luthuli central hospital": "Inkosi Albert Luthuli Central Hospital",
    
    # Kalafong Hospital variations
    "KALAFONG HOSPITAL": "Kalafong Hospital",
    "Kalafong Hospita": "Kalafong Hospital",
    "Kalafong Hospital": "Kalafong Hospital",
    "Kalafong Hospitall": "Kalafong Hospital",
    "kalafong hospital": "Kalafong Hospital",
    
    # King Edward VIII Hospital variations
    "KEH DB": "King Edward VIII Hospital",
    "KEH DBN": "King Edward VIII Hospital",
    "KEH DBNN": "King Edward VIII Hospital",
    "KING EDWARD HOSP": "King Edward VIII Hospital",
    "KING EDWARD VIII HOSP (DBN)": "King Edward VIII Hospital",
    "King Edward Hos": "King Edward VIII Hospital",
    "King Edward Hosp": "King Edward VIII Hospital",
    "King Edward Hospp": "King Edward VIII Hospital",
    "King Edward VIII Hosp (DBN": "King Edward VIII Hospital",
    "King Edward VIII Hosp (DBN)": "King Edward VIII Hospital",
    "King Edward VIII Hosp (DBN))": "King Edward VIII Hospital",
    "keh dbn": "King Edward VIII Hospital",
    "king edward hosp": "King Edward VIII Hospital",
    "king edward viii hosp (dbn)": "King Edward VIII Hospital",
    
    # Khayelitsha CHC variations
    "KHAYELITSHA CHC": "Khayelitsha CHC",
    "KHAYELITSHA SITE B CHC": "Khayelitsha CHC",
    "Khayalitsha CHC": "Khayelitsha CHC",
    "Khayalitsha Sita B CHC": "Khayelitsha CHC",
    "Khayelitsha CH": "Khayelitsha CHC",
    "Khayelitsha CHC": "Khayelitsha CHC",
    "Khayelitsha CHCC": "Khayelitsha CHC",
    "Khayelitsha Site B CH": "Khayelitsha CHC",
    "Khayelitsha Site B CHC": "Khayelitsha CHC",
    "Khayelitsha Site B CHCC": "Khayelitsha CHC",
    "khayelitsha chc": "Khayelitsha CHC",
    "khayelitsha site b chc": "Khayelitsha CHC",
    
    # Letaba Hospital variations
    "LETABA HOSPITAL": "Letaba Hospital",
    "Letaba Hospita": "Letaba Hospital",
    "Letaba Hospital": "Letaba Hospital",
    "Letaba Hospitall": "Letaba Hospital",
    "letaba hospital": "Letaba Hospital",
    
    # Livingstone Hospital variations
    "LIVINGSTONE HOSPITAL (GQEBERHA)": "Livingstone Hospital",
    "Livingstone Hospital": "Livingstone Hospital",
    "Livingstone Hospital (Gqeberha": "Livingstone Hospital",
    "Livingstone Hospital (Gqeberha)": "Livingstone Hospital",
    "Livingstone Hospital (Gqeberha))": "Livingstone Hospital",
    "livingstone hospital (gqeberha)": "Livingstone Hospital",
    
    # Mamelodi Clinic variations
    "MAMELODI CLINIC": "Mamelodi Clinic",
    "MAMELODI CLININC": "Mamelodi Clinic",
    "MAMELODI DAY CLINIC": "Mamelodi Clinic",
    "Mamelodi Clini": "Mamelodi Clinic",
    "Mamelodi Clinic": "Mamelodi Clinic",
    "Mamelodi Clinicc": "Mamelodi Clinic",
    "Mamelodi Clinin": "Mamelodi Clinic",
    "Mamelodi Clininc": "Mamelodi Clinic",
    "Mamelodi Clinincc": "Mamelodi Clinic",
    "Mamelodi Day Clini": "Mamelodi Clinic",
    "Mamelodi Day Clinic": "Mamelodi Clinic",
    "Mamelodi Day Clinicc": "Mamelodi Clinic",
    "mamelodi clinic": "Mamelodi Clinic",
    "mamelodi clininc": "Mamelodi Clinic",
    "mamelodi day clinic": "Mamelodi Clinic",
    
    # Mankweng Hospital variations
    "MANKWENG HOSPITAL": "Mankweng Hospital",
    "Mankweng Hospita": "Mankweng Hospital",
    "Mankweng Hospital": "Mankweng Hospital",
    "Mankweng Hospitall": "Mankweng Hospital",
    "mankweng hospital": "Mankweng Hospital",
    
    # Mitchells Plain Hospital variations
    "MITCHELLS PLAIN HOSPITAL": "Mitchells Plain Hospital",
    "Mitchells Plain Hospita": "Mitchells Plain Hospital",
    "Mitchells Plain Hospital": "Mitchells Plain Hospital",
    "Mitchells Plain Hospitall": "Mitchells Plain Hospital",
    "mitchells plain hospital": "Mitchells Plain Hospital",
    
    # Natalspruit Hospital variations
    "NATALSPRUIT HOSPITAL": "Natalspruit Hospital",
    "Natalspruit Hospita": "Natalspruit Hospital",
    "Natalspruit Hospitall": "Natalspruit Hospital",
    "natalspruit hospital": "Natalspruit Hospital",
    
    # Nelson Mandela Academic Hospital variations
    "NELSON MANDELA ACADEMIC HOSPITAL": "Nelson Mandela Academic Hospital",
    "Nelson Mandela Academic Hospita": "Nelson Mandela Academic Hospital",
    "Nelson Mandela Academic Hospital": "Nelson Mandela Academic Hospital",
    "Nelson Mandela Academic Hospitall": "Nelson Mandela Academic Hospital",
    "nelson mandela academic hospital": "Nelson Mandela Academic Hospital",
    
    # Polokwane Provincial Hospital variations
    "POLOKWANE PROVINCIAL HOSPITAL": "Polokwane Provincial Hospital",
    "Polokwane Provincial Hospita": "Polokwane Provincial Hospital",
    "Polokwane Provincial Hospital": "Polokwane Provincial Hospital",
    "Polokwane Provincial Hospitall": "Polokwane Provincial Hospital",
    "polokwane provincial hospital": "Polokwane Provincial Hospital",
    
    # Prince Mshiyeni Memorial Hospital variations
    "PRINCE MSHIYENI MEMORIAL HOSPITAL": "Prince Mshiyeni Memorial Hospital",
    "Prince Mshiyeni Memorial Hospita": "Prince Mshiyeni Memorial Hospital",
    "Prince Mshiyeni Memorial Hospital": "Prince Mshiyeni Memorial Hospital",
    "Prince Mshiyeni Memorial Hospitall": "Prince Mshiyeni Memorial Hospital",
    "prince mshiyeni memorial hospital": "Prince Mshiyeni Memorial Hospital",
    
    # RK Khan Hospital variations
    "R K KHAN HOSPITAL": "RK Khan Hospital",
    "R K Khan Hospita": "RK Khan Hospital",
    "R K Khan Hospitall": "RK Khan Hospital",
    "RK Khan Hospital": "RK Khan Hospital",
    "r k khan hospital": "RK Khan Hospital",
    
    # Red Cross Childrens Hospital variations
    "RED CROSS CHILDRENS HOSPITAL": "Red Cross Childrens Hospital",
    "Red Cross Childrens Hospita": "Red Cross Childrens Hospital",
    "Red Cross Childrens Hospital": "Red Cross Childrens Hospital",
    "Red Cross Childrens Hospitall": "Red Cross Childrens Hospital",
    "red cross childrens hospital": "Red Cross Childrens Hospital",
    
    # Steve Biko Academic Hospital variations
    "S BIKO ACADEMIC": "Steve Biko Academic Hospital",
    "S Biko Academic": "Steve Biko Academic Hospital",
    "STEVE BIKO ACADEMIC HOSP": "Steve Biko Academic Hospital",
    "STEVE BIKO HOSP": "Steve Biko Academic Hospital",
    "Steve Biko Academic Hos": "Steve Biko Academic Hospital",
    "Steve Biko Academic Hosp": "Steve Biko Academic Hospital",
    "Steve Biko Academic Hospital": "Steve Biko Academic Hospital",
    "Steve Biko Academic Hospp": "Steve Biko Academic Hospital",
    "Steve Biko Hos": "Steve Biko Academic Hospital",
    "Steve Biko Hosp": "Steve Biko Academic Hospital",
    "Steve Biko Hospp": "Steve Biko Academic Hospital",
    "s biko academic": "Steve Biko Academic Hospital",
    "steve biko academic hosp": "Steve Biko Academic Hospital",
    "steve biko hosp": "Steve Biko Academic Hospital",
    
    # Sebokeng Hospital variations
    "SEBOKENG HOSPITAL": "Sebokeng Hospital",
    "Sabokang Hospital": "Sebokeng Hospital",
    "Sebokeng Hospita": "Sebokeng Hospital",
    "Sebokeng Hospital": "Sebokeng Hospital",
    "Sebokeng Hospitall": "Sebokeng Hospital",
    "sebokeng hospital": "Sebokeng Hospital",
    
    # Seshego Hospital variations
    "SESHEGO HOSPITAL": "Seshego Hospital",
    "Sashago Hospital": "Seshego Hospital",
    "Seshego Hospita": "Seshego Hospital",
    "Seshego Hospital": "Seshego Hospital",
    "Seshego Hospitall": "Seshego Hospital",
    "seshego hospital": "Seshego Hospital",
    
    # Site B Clinic variations
    "Sita B Clinic": "Site B Clinic",
    "Site B Clini": "Site B Clinic",
    "Site B Clinic": "Site B Clinic",
    "Site B Clinicc": "Site B Clinic",
    "site b clinic": "Site B Clinic",
    
    # Tembisa Hospital variations
    "TEMBISA HOSPITAL": "Tembisa Hospital",
    "Tembisa Hospita": "Tembisa Hospital",
    "Tembisa Hospital": "Tembisa Hospital",
    "Tembisa Hospitall": "Tembisa Hospital",
    "tembisa hospital": "Tembisa Hospital",
    
    # Tshilidzini Hospital variations
    "TSHILIDZINI HOSPITAL": "Tshilidzini Hospital",
    "Tshilidzini Hospita": "Tshilidzini Hospital",
    "Tshilidzini Hospital": "Tshilidzini Hospital",
    "Tshilidzini Hospitall": "Tshilidzini Hospital",
    "tshilidzini hospital": "Tshilidzini Hospital",
    
    # Tygerberg Hospital variations
    "TYGERBERG HOSP": "Tygerberg Hospital",
    "TYGERBERG HOSPITAL": "Tygerberg Hospital",
    "TYGEREBERG": "Tygerberg Hospital",
    "TYGEREBERG HOSPITL": "Tygerberg Hospital",
    "Tygerberg Hos": "Tygerberg Hospital",
    "Tygerberg Hosp": "Tygerberg Hospital",
    "Tygerberg Hospita": "Tygerberg Hospital",
    "Tygerberg Hospital": "Tygerberg Hospital",
    "Tygerberg Hospitall": "Tygerberg Hospital",
    "Tygerberg Hospp": "Tygerberg Hospital",
    "tygerberg hosp": "Tygerberg Hospital",
    "tygerberg hospital": "Tygerberg Hospital",
    "tygereberg": "Tygerberg Hospital",
    "tygereberg hospitl": "Tygerberg Hospital",
    
    # Uitenhage Provincial Hospital variations
    "UITENHAGE PROVINCIAL HOSPITAL": "Uitenhage Provincial Hospital",
    "Uitenhage Provincial Hospita": "Uitenhage Provincial Hospital",
    "Uitenhage Provincial Hospital": "Uitenhage Provincial Hospital",
    "Uitenhage Provincial Hospitall": "Uitenhage Provincial Hospital",
    "uitenhage provincial hospital": "Uitenhage Provincial Hospital",
    
    # Umlazi Clinic variations
    "UMLAZI CLINIC": "Umlazi Clinic",
    "UMLAZI MEGA": "Umlazi Mega Clinic",
    "UMLAZI MEGA CLINIC": "Umlazi Mega Clinic",
    "Umlazi Clini": "Umlazi Clinic",
    "Umlazi Clinic": "Umlazi Clinic",
    "Umlazi Clinicc": "Umlazi Clinic",
    "Umlazi Maga": "Umlazi Mega Clinic",
    "Umlazi Maga Clinic": "Umlazi Mega Clinic",
    "Umlazi Meg": "Umlazi Mega Clinic",
    "Umlazi Mega": "Umlazi Mega Clinic",
    "Umlazi Mega Clini": "Umlazi Mega Clinic",
    "Umlazi Mega Clinic": "Umlazi Mega Clinic",
    "Umlazi Mega Clinicc": "Umlazi Mega Clinic",
    "Umlazi Megaa": "Umlazi Mega Clinic",
    "umlazi clinic": "Umlazi Clinic",
    "umlazi mega": "Umlazi Mega Clinic",
    "umlazi mega clinic": "Umlazi Mega Clinic",
    
    # Victoria Hospital variations
    "VICTORIA HOSPITAL": "Victoria Hospital",
    "Victoria Hospita": "Victoria Hospital",
    "Victoria Hospital": "Victoria Hospital",
    "Victoria Hospitall": "Victoria Hospital",
    "victoria hospital": "Victoria Hospital",
}

# Enhanced function for better matching
def standardize_facility_name_enhanced(input_name):
    """Convert any facility variation to its canonical name with enhanced matching"""
    if not input_name or not isinstance(input_name, str):
        return input_name
    
    cleaned_input = input_name.strip()
    
    # Try exact match first
    if cleaned_input in facility_map:
        return facility_map[cleaned_input]
    
    # Try case-insensitive match
    lower_input = cleaned_input.lower()
    for key, value in facility_map.items():
        if key.lower() == lower_input:
            return value
    
    # Try partial matching for common patterns
    if 'addington' in lower_input:
        return 'Addington Hospital'
    if 'baragwanath' in lower_input or 'chris hani' in lower_input:
        return 'Chris Hani Baragwanath Hospital'
    if 'charlotte' in lower_input or 'maxeke' in lower_input:
        return 'Charlotte Maxeke Hospital'
    if 'diepkloof' in lower_input:
        return 'Diepkloof Clinic'
    if 'dora' in lower_input and 'nginza' in lower_input:
        return 'Dora Nginza Hospital'
    if 'frere' in lower_input:
        return 'Frere Hospital'
    if 'greys' in lower_input:
        return 'Greys Hospital'
    if 'groote' in lower_input or 'schuur' in lower_input or 'gsh' in lower_input:
        return 'Groote Schuur Hospital'
    if 'gugulethu' in lower_input or 'gugs' in lower_input:
        return 'Gugulethu CHC'
    if 'helen' in lower_input and 'joseph' in lower_input:
        return 'Helen Joseph Hospital'
    if 'inkosi' in lower_input or 'luthuli' in lower_input:
        return 'Inkosi Albert Luthuli Central Hospital'
    if 'kalafong' in lower_input:
        return 'Kalafong Hospital'
    if 'king edward' in lower_input or 'keh' in lower_input:
        return 'King Edward VIII Hospital'
    if 'khayelitsha' in lower_input:
        return 'Khayelitsha CHC'
    if 'letaba' in lower_input:
        return 'Letaba Hospital'
    if 'livingstone' in lower_input:
        return 'Livingstone Hospital'
    if 'mamelodi' in lower_input:
        return 'Mamelodi Clinic'
    if 'mankweng' in lower_input:
        return 'Mankweng Hospital'
    if 'mitchells' in lower_input and 'plain' in lower_input:
        return 'Mitchells Plain Hospital'
    if 'natalspruit' in lower_input:
        return 'Natalspruit Hospital'
    if 'nelson' in lower_input and 'mandela' in lower_input:
        return 'Nelson Mandela Academic Hospital'
    if 'polokwane' in lower_input:
        return 'Polokwane Provincial Hospital'
    if 'prince' in lower_input and 'mshiyeni' in lower_input:
        return 'Prince Mshiyeni Memorial Hospital'
    if 'khan' in lower_input and 'r k' in lower_input:
        return 'RK Khan Hospital'
    if 'red cross' in lower_input:
        return 'Red Cross Childrens Hospital'
    if 'biko' in lower_input:
        return 'Steve Biko Academic Hospital'
    if 'sebokeng' in lower_input:
        return 'Sebokeng Hospital'
    if 'seshego' in lower_input:
        return 'Seshego Hospital'
    if 'site b' in lower_input:
        return 'Site B Clinic'
    if 'tembisa' in lower_input:
        return 'Tembisa Hospital'
    if 'tshilidzini' in lower_input:
        return 'Tshilidzini Hospital'
    if 'tygerberg' in lower_input or 'tygereberg' in lower_input:
        return 'Tygerberg Hospital'
    if 'uitenhage' in lower_input:
        return 'Uitenhage Provincial Hospital'
    if 'umlazi' in lower_input:
        if 'mega' in lower_input:
            return 'Umlazi Mega Clinic'
        return 'Umlazi Clinic'
    if 'victoria' in lower_input:
        return 'Victoria Hospital'
    
    return cleaned_input  # Return original if no good match found

# Apply the enhanced facility standardization
df_cleaned['Facility_Name'] = df_cleaned['Facility_Name'].apply(standardize_facility_name_enhanced)

# Re-analyze the results
print("ANALYZING YOUR DATA AFTER ENHANCED FACILITY CLEANING:")
analyze_actual_facility_data(df_cleaned)

# Show value counts of the cleaned facilities
print("\nCLEANED FACILITY VALUE COUNTS:")
print(df_cleaned['Facility_Name'].value_counts().head(100).sort_index())

ANALYZING YOUR DATA AFTER ENHANCED FACILITY CLEANING:
UNIQUE FACILITY NAMES - BEFORE STANDARDIZATION
 1. Addington Hospital
 2. CHB Hospital
 3. Charlotta Maxaka Hospital
 4. Charlotte Maxeke Hospital
 5. Chris Hani Baragwanath Hospital
 6. Diapkloof Clinic
 7. Diapkloof Clininc
 8. Diepkloof Clinic
 9. Dora Nginza Hospital
10. Frara Hospital
11. Frere Hospital
12. Grays Hospital
13. Greys Hospital
14. Groote Schuur Hospital
15. Gugulathu Clinic
16. Gugulethu CHC
17. Halan Josaph Hospital
18. Helen Joseph Hospital
19. Inkosi Albert Luthuli Central Hospital
20. Kalafong Hospital
21. Khayelitsha CHC
22. King Edward VIII Hospital
23. Lataba Hospital
24. Letaba Hospital
25. Livingstona Hospital (Gqabarha)
26. Livingstone Hospital
27. Mamalodi Clinic
28. Mamalodi Clininc
29. Mamalodi Day Clinic
30. Mamelodi Clinic
31. Mankwang Hospital
32. Mankweng Hospital
33. Mitchalls Plain Hospital
34. Mitchells Plain Hospital
35. Nalson Mandala Acadamic Hospital
36. Natalspruit Hospital
37. Nelson Mand

In [323]:
# Comprehensive facility mapping that includes all the variations from your data
facility_map = {
    # Addington Hospital
    "Addington Hospital": "Addington Hospital",
    
    # CHB Hospital (assuming this is Chris Hani Baragwanath or needs separate mapping)
    "CHB Hospital": "Chris Hani Baragwanath Hospital",
    
    # Charlotte Maxeke Hospital variations
    "Charlotta Maxaka Hospital": "Charlotte Maxeke Hospital",
    "Charlotte Maxeke Hospital": "Charlotte Maxeke Hospital",
    
    # Chris Hani Baragwanath Hospital
    "Chris Hani Baragwanath Hospital": "Chris Hani Baragwanath Hospital",
    
    # Diepkloof Clinic variations
    "Diapkloof Clinic": "Diepkloof Clinic",
    "Diapkloof Clininc": "Diepkloof Clinic",
    "Diepkloof Clinic": "Diepkloof Clinic",
    
    # Dora Nginza Hospital
    "Dora Nginza Hospital": "Dora Nginza Hospital",
    
    # Frere Hospital variations
    "Frara Hospital": "Frere Hospital",
    "Frere Hospital": "Frere Hospital",
    
    # Greys Hospital variations
    "Grays Hospital": "Greys Hospital",
    "Greys Hospital": "Greys Hospital",
    
    # Groote Schuur Hospital
    "Groote Schuur Hospital": "Groote Schuur Hospital",
    
    # Gugulethu variations
    "Gugulathu Clinic": "Gugulethu CHC",
    "Gugulethu CHC": "Gugulethu CHC",
    
    # Helen Joseph Hospital variations
    "Halan Josaph Hospital": "Helen Joseph Hospital",
    "Helen Joseph Hospital": "Helen Joseph Hospital",
    
    # Inkosi Albert Luthuli Central Hospital
    "Inkosi Albert Luthuli Central Hospital": "Inkosi Albert Luthuli Central Hospital",
    
    # Kalafong Hospital
    "Kalafong Hospital": "Kalafong Hospital",
    
    # Khayelitsha CHC
    "Khayelitsha CHC": "Khayelitsha CHC",
    
    # King Edward VIII Hospital
    "King Edward VIII Hospital": "King Edward VIII Hospital",
    
    # Letaba Hospital variations
    "Lataba Hospital": "Letaba Hospital",
    "Letaba Hospital": "Letaba Hospital",
    
    # Livingstone Hospital variations
    "Livingstona Hospital (Gqabarha)": "Livingstone Hospital",
    "Livingstone Hospital": "Livingstone Hospital",
    
    # Mamelodi Clinic variations
    "Mamalodi Clinic": "Mamelodi Clinic",
    "Mamalodi Clininc": "Mamelodi Clinic",
    "Mamalodi Day Clinic": "Mamelodi Clinic",
    "Mamelodi Clinic": "Mamelodi Clinic",
    
    # Mankweng Hospital variations
    "Mankwang Hospital": "Mankweng Hospital",
    "Mankweng Hospital": "Mankweng Hospital",
    
    # Mitchells Plain Hospital variations
    "Mitchalls Plain Hospital": "Mitchells Plain Hospital",
    "Mitchells Plain Hospital": "Mitchells Plain Hospital",
    
    # Nelson Mandela variations
    "Nalson Mandala Acadamic Hospital": "Nelson Mandela Academic Hospital",
    "Natalspruit Hospital": "Nelson Mandela Academic Hospital",
    "Nelson Mandela Academic Hospital": "Nelson Mandela Academic Hospital",
    
    # Polokwane variations
    "Polokwana Provincial Hospital": "Polokwane Provincial Hospital",
    "Polokwane Provincial Hospital": "Polokwane Provincial Hospital",
    
    # Prince Mshiyeni variations
    "Princa Mshiyani Mamorial Hospital": "Prince Mshiyeni Memorial Hospital",
    "Prince Mshiyeni Memorial Hospital": "Prince Mshiyeni Memorial Hospital",
    
    # RK Khan Hospital
    "RK Khan Hospital": "RK Khan Hospital",
    
    # Red Cross variations
    "Rad Cross Childrans Hospital": "Red Cross Childrens Hospital",
    "Red Cross Childrens Hospital": "Red Cross Childrens Hospital",
    
    # Sebokeng Hospital
    "Sebokeng Hospital": "Sebokeng Hospital",
    
    # Seshego Hospital
    "Seshego Hospital": "Seshego Hospital",
    
    # Site B Clinic
    "Site B Clinic": "Site B Clinic",
    
    # Steve Biko variations
    "Steve Biko Academic Hospital": "Steve Biko Academic Hospital",
    
    # Tembisa variations
    "Tambisa Hospital": "Tembisa Hospital",
    "Tembisa Hospital": "Tembisa Hospital",
    
    # Tshilidzini Hospital
    "Tshilidzini Hospital": "Tshilidzini Hospital",
    
    # Tygerberg variations
    "Tygarabarg": "Tygerberg Hospital",
    "Tygarabarg Hospitl": "Tygerberg Hospital",
    "Tygarbarg Hosp": "Tygerberg Hospital",
    "Tygarbarg Hospital": "Tygerberg Hospital",
    "Tygerberg Hospital": "Tygerberg Hospital",
    "Tygereber": "Tygerberg Hospital",
    
    # Uitenhage variations
    "Uitanhaga Provincial Hospital": "Uitenhage Provincial Hospital",
    "Uitenhage Provincial Hospital": "Uitenhage Provincial Hospital",
    
    # Umlazi variations
    "Umlazi Clinic": "Umlazi Clinic",
    "Umlazi Mega Clinic": "Umlazi Mega Clinic",
    
    # Victoria Hospital
    "Victoria Hospital": "Victoria Hospital",
}

# Enhanced function with better matching
def standardize_facility_name_comprehensive(input_name):
    """Convert any facility variation to its canonical name with comprehensive matching"""
    if not input_name or not isinstance(input_name, str):
        return input_name
    
    cleaned_input = input_name.strip()
    
    # Try exact match first
    if cleaned_input in facility_map:
        return facility_map[cleaned_input]
    
    # Try case-insensitive match
    lower_input = cleaned_input.lower()
    for key, value in facility_map.items():
        if key.lower() == lower_input:
            return value
    
    # Enhanced partial matching with fuzzy logic
    if 'addington' in lower_input:
        return 'Addington Hospital'
    if 'chb' in lower_input:
        return 'Chris Hani Baragwanath Hospital'
    if 'charlotte' in lower_input or 'maxeke' in lower_input or 'maxaka' in lower_input:
        return 'Charlotte Maxeke Hospital'
    if 'chris' in lower_input or 'baragwanath' in lower_input:
        return 'Chris Hani Baragwanath Hospital'
    if 'diepkloof' in lower_input or 'diapkloof' in lower_input:
        return 'Diepkloof Clinic'
    if 'dora' in lower_input and 'nginza' in lower_input:
        return 'Dora Nginza Hospital'
    if 'frere' in lower_input or 'frara' in lower_input:
        return 'Frere Hospital'
    if 'grey' in lower_input:
        return 'Greys Hospital'
    if 'groote' in lower_input or 'schuur' in lower_input:
        return 'Groote Schuur Hospital'
    if 'gugulethu' in lower_input or 'gugulathu' in lower_input:
        return 'Gugulethu CHC'
    if 'helen' in lower_input or 'joseph' in lower_input or 'halan' in lower_input or 'josaph' in lower_input:
        return 'Helen Joseph Hospital'
    if 'inkosi' in lower_input or 'luthuli' in lower_input:
        return 'Inkosi Albert Luthuli Central Hospital'
    if 'kalafong' in lower_input:
        return 'Kalafong Hospital'
    if 'khayelitsha' in lower_input:
        return 'Khayelitsha CHC'
    if 'king' in lower_input and 'edward' in lower_input:
        return 'King Edward VIII Hospital'
    if 'letaba' in lower_input or 'lataba' in lower_input:
        return 'Letaba Hospital'
    if 'livingstone' in lower_input:
        return 'Livingstone Hospital'
    if 'mamelodi' in lower_input or 'mamalodi' in lower_input:
        return 'Mamelodi Clinic'
    if 'mankweng' in lower_input or 'mankwang' in lower_input:
        return 'Mankweng Hospital'
    if 'mitchell' in lower_input or 'mitchall' in lower_input:
        return 'Mitchells Plain Hospital'
    if 'nelson' in lower_input or 'mandela' in lower_input or 'natalspruit' in lower_input or 'nalson' in lower_input:
        return 'Nelson Mandela Academic Hospital'
    if 'polokwane' in lower_input or 'polokwana' in lower_input:
        return 'Polokwane Provincial Hospital'
    if 'prince' in lower_input or 'mshiyeni' in lower_input or 'princa' in lower_input or 'mshiyani' in lower_input:
        return 'Prince Mshiyeni Memorial Hospital'
    if 'khan' in lower_input:
        return 'RK Khan Hospital'
    if 'red cross' in lower_input or 'rad cross' in lower_input:
        return 'Red Cross Childrens Hospital'
    if 'sebokeng' in lower_input:
        return 'Sebokeng Hospital'
    if 'seshego' in lower_input:
        return 'Seshego Hospital'
    if 'site b' in lower_input:
        return 'Site B Clinic'
    if 'steve' in lower_input or 'biko' in lower_input:
        return 'Steve Biko Academic Hospital'
    if 'tembisa' in lower_input or 'tambisa' in lower_input:
        return 'Tembisa Hospital'
    if 'tshilidzini' in lower_input:
        return 'Tshilidzini Hospital'
    if 'tygerberg' in lower_input or 'tygarabarg' in lower_input or 'tygereber' in lower_input or 'tygarbarg' in lower_input:
        return 'Tygerberg Hospital'
    if 'uitenhage' in lower_input or 'uitanhaga' in lower_input:
        return 'Uitenhage Provincial Hospital'
    if 'umlazi' in lower_input:
        if 'mega' in lower_input:
            return 'Umlazi Mega Clinic'
        return 'Umlazi Clinic'
    if 'victoria' in lower_input:
        return 'Victoria Hospital'
    
    return cleaned_input  # Return original if no good match found

# Apply the comprehensive standardization
df_cleaned['Facility_Name'] = df_cleaned['Facility_Name'].apply(standardize_facility_name_comprehensive)

# Re-analyze the results
print("ANALYZING YOUR DATA AFTER COMPREHENSIVE FACILITY CLEANING:")
analyze_actual_facility_data(df_cleaned)

# Show value counts of the cleaned facilities
print("\nCLEANED FACILITY VALUE COUNTS:")
print(df_cleaned['Facility_Name'].value_counts().head(20))

# Check for any remaining unmapped values
def check_remaining_variations(df, column_name='Facility_Name'):
    """Check for any remaining variations that might need manual mapping"""
    unique_values = df[column_name].unique()
    print(f"\nRemaining unique facility names: {len(unique_values)}")
    for value in sorted(unique_values):
        print(f"'{value}'")

check_remaining_variations(df_cleaned)

ANALYZING YOUR DATA AFTER COMPREHENSIVE FACILITY CLEANING:
UNIQUE FACILITY NAMES - BEFORE STANDARDIZATION
 1. Addington Hospital
 2. Charlotte Maxeke Hospital
 3. Chris Hani Baragwanath Hospital
 4. Diepkloof Clinic
 5. Dora Nginza Hospital
 6. Frere Hospital
 7. Greys Hospital
 8. Groote Schuur Hospital
 9. Gugulethu CHC
10. Helen Joseph Hospital
11. Inkosi Albert Luthuli Central Hospital
12. Kalafong Hospital
13. Khayelitsha CHC
14. King Edward VIII Hospital
15. Letaba Hospital
16. Livingstone Hospital
17. Mamelodi Clinic
18. Mankweng Hospital
19. Mitchells Plain Hospital
20. Nelson Mandela Academic Hospital
21. Polokwane Provincial Hospital
22. Prince Mshiyeni Memorial Hospital
23. RK Khan Hospital
24. Red Cross Childrens Hospital
25. Sebokeng Hospital
26. Seshego Hospital
27. Site B Clinic
28. Steve Biko Academic Hospital
29. Tembisa Hospital
30. Tshilidzini Hospital
31. Tygerberg Hospital
32. Uitenhage Provincial Hospital
33. Umlazi Clinic
34. Umlazi Mega Clinic
35. Victoria Hospi

In [324]:
# Comprehensive complaint mapping
complaint_map = {
    # Allergic Reaction variations
    "ALLERGIC REACTION": "Allergic Reaction",
    "allargic raaction": "Allergic Reaction",
    "allergic reactio": "Allergic Reaction",
    "allergic reaction": "Allergic Reaction",
    "allergic reactionn": "Allergic Reaction",
    
    # Animal Bite variations
    "ANIMAL BITE": "Animal Bite",
    "animal bit": "Animal Bite",
    "animal bita": "Animal Bite",
    "animal bite": "Animal Bite",
    "animal bitee": "Animal Bite",
    
    # Asthma Attack variations
    "ASTHMA ATTACK": "Asthma Attack",
    "asthma attac": "Asthma Attack",
    "asthma attack": "Asthma Attack",
    "asthma attackk": "Asthma Attack",
    
    # Back Pain variations
    "BACK PAIN": "Back Pain",
    "back pai": "Back Pain",
    "back pain": "Back Pain",
    "back painn": "Back Pain",
    
    # Burns variations
    "BURNS": "Burns",
    "burn": "Burns",
    "burns": "Burns",
    "burnss": "Burns",
    
    # Chest Pain variations
    "CHEST PAIN": "Chest Pain",
    "chast pain": "Chest Pain",
    "chest pai": "Chest Pain",
    "chest pain": "Chest Pain",
    "chest painn": "Chest Pain",
    
    # Child Not Feeding variations
    "CHILD NOT FEEDING": "Child Not Feeding",
    "child not faading": "Child Not Feeding",
    "child not feedin": "Child Not Feeding",
    "child not feeding": "Child Not Feeding",
    "child not feedingg": "Child Not Feeding",
    
    # Cough & Fever variations
    "COUGH & FEVER": "Cough & Fever",
    "Cough & favar": "Cough & Fever",
    "Cough & feve": "Cough & Fever",
    "Cough & fever": "Cough & Fever",
    "Cough & feverr": "Cough & Fever",
    "cough & fever": "Cough & Fever",
    
    # Dental Pain variations
    "DENTAL PAIN": "Dental Pain",
    "dantal pain": "Dental Pain",
    "dental pai": "Dental Pain",
    "dental pain": "Dental Pain",
    "dental painn": "Dental Pain",
    
    # Diabetes Checkup variations
    "DIABETES CHECKUP": "Diabetes Checkup",
    "diabatas chackup": "Diabetes Checkup",
    "diabetes checku": "Diabetes Checkup",
    "diabetes checkup": "Diabetes Checkup",
    "diabetes checkupp": "Diabetes Checkup",
    
    # Diarrhoea variations
    "DIARRHOEA": "Diarrhoea",
    "diarrhoaa": "Diarrhoea",
    "diarrhoe": "Diarrhoea",
    "diarrhoea": "Diarrhoea",
    "diarrhoeaa": "Diarrhoea",
    
    # Dizziness variations
    "DIZZINESS": "Dizziness",
    "dizzinass": "Dizziness",
    "dizzines": "Dizziness",
    "dizziness": "Dizziness",
    "dizzinesss": "Dizziness",
    
    # Eye Problem variations
    "EYE PROBLEM": "Eye Problem",
    "aya problam": "Eye Problem",
    "eye proble": "Eye Problem",
    "eye problem": "Eye Problem",
    "eye problemm": "Eye Problem",
    
    # Fractured Arm variations
    "FRACTURED ARM": "Fractured Arm",
    "fracturad arm": "Fractured Arm",
    "fractured ar": "Fractured Arm",
    "fractured arm": "Fractured Arm",
    "fractured armm": "Fractured Arm",
    
    # Headache variations
    "HEADACHE": "Headache",
    "haadacha": "Headache",
    "headach": "Headache",
    "headache": "Headache",
    "headachee": "Headache",
    
    # Hypertension Review variations
    "HYPERTENSION REVIEW": "Hypertension Review",
    "hypartansion raviaw": "Hypertension Review",
    "hypertension revie": "Hypertension Review",
    "hypertension review": "Hypertension Review",
    "hypertension revieww": "Hypertension Review",
    
    # Mental Health Crisis variations
    "MENTAL HEALTH CRISIS": "Mental Health Crisis",
    "mantal haalth crisis": "Mental Health Crisis",
    "mental health crisi": "Mental Health Crisis",
    "mental health crisis": "Mental Health Crisis",
    "mental health crisiss": "Mental Health Crisis",
    
    # MVA Passenger variations
    "MVA PASSENGER": "MVA Passenger",
    "MVA passangar": "MVA Passenger",
    "MVA passenge": "MVA Passenger",
    "MVA passenger": "MVA Passenger",
    "MVA passengerr": "MVA Passenger",
    "mva passenger": "MVA Passenger",
    
    # Rash variations
    "RASH": "Rash",
    "ras": "Rash",
    "rash": "Rash",
    "rashh": "Rash",
    
    # Routine Antenatal variations
    "ROUTINE ANTENATAL": "Routine Antenatal",
    "routina antanatal": "Routine Antenatal",
    "routine antenata": "Routine Antenatal",
    "routine antenatal": "Routine Antenatal",
    "routine antenatall": "Routine Antenatal",
    
    # Seizure variations
    "SEIZURE": "Seizure",
    "saizura": "Seizure",
    "seizur": "Seizure",
    "seizure": "Seizure",
    "seizuree": "Seizure",
    
    # Severe Abd Pain variations
    "SEVERE ABD PAIN": "Severe Abd Pain",
    "savara abd pain": "Severe Abd Pain",
    "severe abd pai": "Severe Abd Pain",
    "severe abd pain": "Severe Abd Pain",
    "severe abd painn": "Severe Abd Pain",
    
    # Shortness of Breath variations
    "SHORTNESS OF BREATH": "Shortness of Breath",
    "shortnass of braath": "Shortness of Breath",
    "shortness of breat": "Shortness of Breath",
    "shortness of breath": "Shortness of Breath",
    "shortness of breathh": "Shortness of Breath",
    
    # Stab Wound Chest variations
    "STAB WOUND CHEST": "Stab Wound Chest",
    "stab wound chast": "Stab Wound Chest",
    "stab wound ches": "Stab Wound Chest",
    "stab wound chest": "Stab Wound Chest",
    "stab wound chestt": "Stab Wound Chest",
    
    # STI Screening variations
    "STI SCREENING": "STI Screening",
    "STI scraaning": "STI Screening",
    "STI screenin": "STI Screening",
    "STI screening": "STI Screening",
    "STI screeningg": "STI Screening",
    "sti screening": "STI Screening",
    
    # TB Medication Collection variations
    "TB MEDS COLLECTION": "TB Medication Collection",
    "TB mads collaction": "TB Medication Collection",
    "TB meds collectio": "TB Medication Collection",
    "TB meds collection": "TB Medication Collection",
    "TB meds collectionn": "TB Medication Collection",
    "tb meds collection": "TB Medication Collection",
    
    # UTI Symptoms variations
    "UTI SYMPTOMS": "UTI Symptoms",
    "UTI symptom": "UTI Symptoms",
    "UTI symptoms": "UTI Symptoms",
    "UTI symptomss": "UTI Symptoms",
    "uti symptoms": "UTI Symptoms",
    
    # Vomiting variations
    "VOMITING": "Vomiting",
    "vomitin": "Vomiting",
    "vomiting": "Vomiting",
    "vomitingg": "Vomiting",
}

# Enhanced function for better matching
def standardize_complaint_name_comprehensive(input_name):
    """Convert any complaint variation to its canonical name with comprehensive matching"""
    if not input_name or not isinstance(input_name, str):
        return input_name
    
    cleaned_input = input_name.strip()
    
    # Try exact match first
    if cleaned_input in complaint_map:
        return complaint_map[cleaned_input]
    
    # Try case-insensitive match
    lower_input = cleaned_input.lower()
    for key, value in complaint_map.items():
        if key.lower() == lower_input:
            return value
    
    # Enhanced partial matching
    if 'allerg' in lower_input:
        return 'Allergic Reaction'
    if 'animal' in lower_input and ('bit' in lower_input or 'bite' in lower_input):
        return 'Animal Bite'
    if 'asthma' in lower_input:
        return 'Asthma Attack'
    if 'back' in lower_input and 'pain' in lower_input:
        return 'Back Pain'
    if 'burn' in lower_input:
        return 'Burns'
    if 'chest' in lower_input and 'pain' in lower_input:
        return 'Chest Pain'
    if 'child' in lower_input and ('feed' in lower_input or 'fad' in lower_input):
        return 'Child Not Feeding'
    if 'cough' in lower_input and 'fever' in lower_input:
        return 'Cough & Fever'
    if ('dental' in lower_input or 'danta' in lower_input) and 'pain' in lower_input:
        return 'Dental Pain'
    if 'diabet' in lower_input and ('check' in lower_input or 'chack' in lower_input):
        return 'Diabetes Checkup'
    if 'diarrh' in lower_input:
        return 'Diarrhoea'
    if 'dizz' in lower_input:
        return 'Dizziness'
    if ('eye' in lower_input or 'aya' in lower_input) and 'problem' in lower_input:
        return 'Eye Problem'
    if 'fractur' in lower_input and 'arm' in lower_input:
        return 'Fractured Arm'
    if 'headach' in lower_input or 'haadach' in lower_input:
        return 'Headache'
    if 'hypertens' in lower_input and 'review' in lower_input:
        return 'Hypertension Review'
    if 'mental' in lower_input and 'health' in lower_input:
        return 'Mental Health Crisis'
    if 'mva' in lower_input and 'passeng' in lower_input:
        return 'MVA Passenger'
    if 'rash' in lower_input or 'ras' in lower_input:
        return 'Rash'
    if 'routine' in lower_input and 'antenat' in lower_input:
        return 'Routine Antenatal'
    if 'seizur' in lower_input or 'saizur' in lower_input:
        return 'Seizure'
    if 'severe' in lower_input and ('abd' in lower_input or 'abdom' in lower_input) and 'pain' in lower_input:
        return 'Severe Abd Pain'
    if 'shortness' in lower_input and 'breath' in lower_input:
        return 'Shortness of Breath'
    if 'stab' in lower_input and 'wound' in lower_input and 'chest' in lower_input:
        return 'Stab Wound Chest'
    if 'sti' in lower_input and 'screen' in lower_input:
        return 'STI Screening'
    if 'tb' in lower_input and ('med' in lower_input or 'mads' in lower_input) and 'collect' in lower_input:
        return 'TB Medication Collection'
    if 'uti' in lower_input and 'symptom' in lower_input:
        return 'UTI Symptoms'
    if 'vomit' in lower_input:
        return 'Vomiting'
    
    return cleaned_input  # Return original if no good match found

# Apply the comprehensive standardization to Chief_Complaint column
df_cleaned['Chief_Complaint'] = df_cleaned['Chief_Complaint'].apply(standardize_complaint_name_comprehensive)

# Re-analyze the results
print("ANALYZING YOUR DATA AFTER COMPREHENSIVE CHIEF COMPLAINT CLEANING:")
analyze_actual_complaint_data(df_cleaned, 'Chief_Complaint')

# Show value counts of the cleaned complaints
print("\nCLEANED CHIEF COMPLAINT VALUE COUNTS:")
print(df_cleaned['Chief_Complaint'].value_counts().head(20))

# Check for any remaining unmapped values
def check_remaining_complaint_variations(df, column_name='Chief_Complaint'):
    """Check for any remaining variations that might need manual mapping"""
    unique_values = df[column_name].unique()
    print(f"\nRemaining unique complaint names: {len(unique_values)}")
    for value in sorted(unique_values):
        print(f"'{value}'")

check_remaining_complaint_variations(df_cleaned)

ANALYZING YOUR DATA AFTER COMPREHENSIVE CHIEF COMPLAINT CLEANING:
UNIQUE COMPLAINT NAMES - BEFORE STANDARDIZATION
 1. Allergic Reaction
 2. Animal Bite
 3. Asthma Attack
 4. Back Pain
 5. Burns
 6. Chest Pain
 7. Child Not Feeding
 8. Cough & Fever
 9. Dental Pain
10. Diabetes Checkup
11. Diarrhoea
12. Dizziness
13. Eye Problem
14. Fractured Arm
15. Headache
16. Hypertension Review
17. MVA Passenger
18. Mental Health Crisis
19. Rash
20. Routine Antenatal
21. STI Screening
22. Seizure
23. Severe Abd Pain
24. Shortness of Breath
25. Stab Wound Chest
26. TB Medication Collection
27. UTI Symptoms
28. Vomiting

UNIQUE COMPLAINT NAMES - AFTER STANDARDIZATION
 1. Allergic Reaction
 2. Animal Bite
 3. Asthma Attack
 4. Back Pain
 5. Burns
 6. Chest Pain
 7. Child Not Feeding
 8. Cough & Fever
 9. Dental Pain
10. Diabetes Checkup
11. Diarrhoea
12. Dizziness
13. Eye Problem
14. Fractured Arm
15. Headache
16. Hypertension Review
17. MVA Passenger
18. Mental Health Crisis
19. Rash
20. Routine Ante

In [325]:
print("Unique counts before cleaning:")
print(df["Department"].nunique(), df["Facility_Name"].nunique(), df["Chief_Complaint"].nunique())

print("\nUnique counts after cleaning:")
print(df_cleaned["Department"].nunique(), df_cleaned["Facility_Name"].nunique(), df_cleaned["Chief_Complaint"].nunique())


Unique counts before cleaning:
155 311 139

Unique counts after cleaning:
11 35 28


In [326]:
def clean_gender_column(series, column_name='Gender'):
    """
    Standardize gender categories and handle missing values
    Returns cleaned series with detailed report 👥
    """
    print(f" Cleaning {column_name} column...")
    
    # Gender mapping with common variations
    gender_mapping = {
        'F': 'Female', 'Female': 'Female', 'fem': 'Female', 'Fem': 'Female', 'f': 'Female',
        'M': 'Male', 'MALE': 'Male', 'Male': 'Male', 'Mal': 'Male', 'm': 'Male',
        'Intersex': 'Intersex', 'Inter': 'Intersex', 'Intersexed': 'Intersex',
        'Other': 'Other', 'O': 'Other', 'Non-binary': 'Other', 'Nonbinary': 'Other',
        'U': 'Unknown', 'Unknown': 'Unknown', 'Unk': 'Unknown', 'N/A': 'Unknown'
    }
    
    def standardize_gender(gender):
        if pd.isna(gender):
            return np.nan
        
        gender_str = str(gender).strip().title()
        return gender_mapping.get(gender_str, 'Unknown')
    
    # Count before processing
    original_unique = series.nunique()
    original_missing = series.isna().sum()
    original_distribution = series.value_counts()
    
    # Apply standardization
    cleaned_series = series.apply(standardize_gender)
    
    # Impute missing values with mode
    mode_value = cleaned_series.mode()[0] if not cleaned_series.mode().empty else 'Unknown'
    cleaned_series = cleaned_series.fillna(mode_value)
    
    # Count after processing
    cleaned_unique = cleaned_series.nunique()
    cleaned_missing = cleaned_series.isna().sum()
    cleaned_distribution = cleaned_series.value_counts()
    
    # Print detailed report
    print(f"    Unique values: {original_unique} → {cleaned_unique}")
    print(f"    Missing values imputed: {original_missing} → {cleaned_missing}")
    print(f"    Final distribution:")
    for category, count in cleaned_distribution.items():
        percentage = (count / len(cleaned_series)) * 100
        print(f"     • {category}: {count} ({percentage:.1f}%)")
    print()
    
    return cleaned_series

def clean_triage_category(series, column_name='Triage_Category'):
    """
    Standardize triage category colors and handle missing values
    Returns cleaned series with detailed report 
    """
    print(f" Cleaning {column_name} column...")
    
    # Triage category mapping with severity levels
    triage_mapping = {
        'Red': 'Red', 'Read': 'Red', 'Rede': 'Red', 'RED': 'Red', 'R': 'Red',        # Immediate
        'Orange': 'Orange', 'Orang': 'Orange', 'Oringe': 'Orange', 'ORANGE': 'Orange', 'O': 'Orange',  # Very Urgent
        'Yellow': 'Yellow', 'Yello': 'Yellow', 'Yelow': 'Yellow', 'YELLOW': 'Yellow', 'Y': 'Yellow',   # Urgent
        'Green': 'Green', 'Grean': 'Green', 'Greeen': 'Green', 'GREEN': 'Green', 'G': 'Green',        # Standard
        'Blue': 'Blue', 'Blew': 'Blue', 'Blu': 'Blue', 'BLUE': 'Blue', 'B': 'Blue'    # Non-urgent
    }
    
    def standardize_triage(triage):
        if pd.isna(triage):
            return np.nan
        
        triage_str = str(triage).strip().title()
        
        # Direct mapping first
        if triage_str in triage_mapping:
            return triage_mapping[triage_str]
        
        # Handle case variations and partial matches
        triage_lower = triage_str.lower()
        if any(word in triage_lower for word in ['red', 'emergency', 'resus']):
            return 'Red'
        elif any(word in triage_lower for word in ['orange', 'very urgent', 'priority 1']):
            return 'Orange'
        elif any(word in triage_lower for word in ['yellow', 'urgent', 'priority 2']):
            return 'Yellow'
        elif any(word in triage_lower for word in ['green', 'standard', 'routine', 'priority 3']):
            return 'Green'
        elif any(word in triage_lower for word in ['blue', 'non-urgent', 'walk-in', 'priority 4']):
            return 'Blue'
        else:
            return 'Unknown'
    
    # Count before processing
    original_unique = series.nunique()
    original_missing = series.isna().sum()
    original_distribution = series.value_counts()
    
    # Apply standardization
    cleaned_series = series.apply(standardize_triage)
    
    # Impute missing values with mode
    mode_value = cleaned_series.mode()[0] if not cleaned_series.mode().empty else 'Unknown'
    cleaned_series = cleaned_series.fillna(mode_value)
    
    # Count after processing
    cleaned_unique = cleaned_series.nunique()
    cleaned_missing = cleaned_series.isna().sum()
    cleaned_distribution = cleaned_series.value_counts()
    
    # Print detailed report
    print(f"    Unique values: {original_unique} → {cleaned_unique}")
    print(f"    Missing values imputed: {original_missing} → {cleaned_missing}")
    print(f"    Final distribution:")
    for category, count in cleaned_distribution.items():
        percentage = (count / len(cleaned_series)) * 100
        
        print(f"     • {category}: {count} ({percentage:.1f}%)")
    print()
    
    return cleaned_series

def clean_final_categorical(df):
    """
    Clean gender and triage columns with comprehensive reporting
    Returns cleaned dataframe with standardized categories 
    """
    df_cleaned = df.copy()
    
    print("="*60)
    print(" Starting gender and triage column cleaning...")
    print("="*60)
    
    # Clean Gender column
    if 'Gender' in df.columns:
        df_cleaned['Gender'] = clean_gender_column(df['Gender'])
    
    # Clean Triage_Category column
    if 'Triage_Category' in df.columns:
        df_cleaned['Triage_Category'] = clean_triage_category(df['Triage_Category'])
    
    return df_cleaned

# --- Execute the cleaning ---
print("\n" + "="*60)
print(" Starting final categorical data cleaning...")
print("="*60)

df_cleaned = clean_final_categorical(df_cleaned)

# --- Final Summary ---
print("="*60)
print(" Gender and triage cleaning completed successfully! ")
print(f" Final dataset shape: {df_cleaned.shape}")

# Show comprehensive final value counts
print("\n FINAL DISTRIBUTION SUMMARY:")
print("-" * 40)

if 'Gender' in df_cleaned.columns:
    print("\n GENDER DISTRIBUTION:")
    gender_counts = df_cleaned['Gender'].value_counts()
    total = len(df_cleaned)
    for gender, count in gender_counts.items():
        percentage = (count / total) * 100
        print(f"  {gender}: {count} ({percentage:.1f}%)")

       


if 'Triage_Category' in df_cleaned.columns:
    print("\n TRIAGE CATEGORY DISTRIBUTION:")
    triage_counts = df_cleaned['Triage_Category'].value_counts()
    for category, count in triage_counts.items():
        percentage = (count / total) * 100
        
        print(f"  {category}: {count} ({percentage:.1f}%)")

print("="*60)


 Starting final categorical data cleaning...
 Starting gender and triage column cleaning...
 Cleaning Gender column...
    Unique values: 9 → 5
    Missing values imputed: 8659 → 0
    Final distribution:
     • Male: 129488 (51.8%)
     • Female: 117211 (46.9%)
     • Unknown: 2365 (0.9%)
     • Intersex: 481 (0.2%)
     • Other: 455 (0.2%)

 Cleaning Triage_Category column...
    Unique values: 20 → 6
    Missing values imputed: 7579 → 0
    Final distribution:
     • Green: 111064 (44.4%)
     • Yellow: 68882 (27.6%)
     • Orange: 34560 (13.8%)
     • Red: 12172 (4.9%)
     • Unknown: 11696 (4.7%)
     • Blue: 11626 (4.7%)

 Gender and triage cleaning completed successfully! 
 Final dataset shape: (250000, 28)

 FINAL DISTRIBUTION SUMMARY:
----------------------------------------

 GENDER DISTRIBUTION:
  Male: 129488 (51.8%)
  Female: 117211 (46.9%)
  Unknown: 2365 (0.9%)
  Intersex: 481 (0.2%)
  Other: 455 (0.2%)

 TRIAGE CATEGORY DISTRIBUTION:
  Green: 111064 (44.4%)
  Yellow: 6

In [ ]:
def clean_day_of_week(series, column_name='Day_of_Week'):
    """
    Standardize day of week categories
    Returns cleaned series with detailed report 
    """
    print(f" Cleaning {column_name} column...")
    
    # Day of week mapping with comprehensive variations
    day_mapping = {
        # Monday variations
        'Mon': 'Monday', 'Monday': 'Monday', 'monday': 'Monday', 'MON': 'Monday',
        'Mon.': 'Monday', 'Mond': 'Monday', 'Mo': 'Monday',
        
        # Tuesday variations
        'Tue': 'Tuesday', 'Tuesday': 'Tuesday', 'tuesday': 'Tuesday', 'TUE': 'Tuesday',
        'Tue.': 'Tuesday', 'Tues': 'Tuesday', 'Tu': 'Tuesday',
        
        # Wednesday variations
        'Wed': 'Wednesday', 'Wednesday': 'Wednesday', 'wednesday': 'Wednesday', 'WED': 'Wednesday',
        'Wed.': 'Wednesday', 'Wednes': 'Wednesday', 'We': 'Wednesday',
        
        # Thursday variations
        'Thu': 'Thursday', 'Thursday': 'Thursday', 'thursday': 'Thursday', 'THU': 'Thursday',
        'Thu.': 'Thursday', 'Thur': 'Thursday', 'Thurs': 'Thursday', 'Th': 'Thursday',
        
        # Friday variations
        'Fri': 'Friday', 'Friday': 'Friday', 'friday': 'Friday', 'FRI': 'Friday',
        'Fri.': 'Friday', 'Fr': 'Friday',
        
        # Saturday variations
        'Sat': 'Saturday', 'Saturday': 'Saturday', 'saturday': 'Saturday', 'SAT': 'Saturday',
        'Sat.': 'Saturday', 'Sa': 'Saturday',
        
        # Sunday variations
        'Sun': 'Sunday', 'Sunday': 'Sunday', 'sunday': 'Sunday', 'SUN': 'Sunday',
        'Sun.': 'Sunday', 'Su': 'Sunday'
    }
    
    def standardize_day(day):
        if pd.isna(day):
            return np.nan
        
        day_str = str(day).strip().title()
        return day_mapping.get(day_str, 'Unknown')
    
    # Count before processing
    original_unique = series.nunique()
    original_missing = series.isna().sum()
    original_distribution = series.value_counts()
    
    # Apply standardization
    cleaned_series = series.apply(standardize_day)
    
    # Print summary
    cleaned_unique = cleaned_series.nunique()
    cleaned_missing = cleaned_series.isna().sum()
    cleaned_distribution = cleaned_series.value_counts()
    
    # Print detailed report
    print(f"    Unique values: {original_unique} → {cleaned_unique}")
    print(f"    Final distribution:")
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'Unknown']
   
    
    for day in day_order:
        if day in cleaned_distribution:
            count = cleaned_distribution[day]
            percentage = (count / len(cleaned_series)) * 100
            print(f"      {day}: {count} ({percentage:.1f}%)")
    print()
    
    return cleaned_series

def clean_final_categorical(df):
    """
    Clean all categorical columns including Day_of_Week
    Returns cleaned dataframe with standardized categories 
    """
    df_cleaned = df.copy()
    
    print("="*60)
    print(" Starting day of week column cleaning...")
    print("="*60)
    
    # Clean Day_of_Week column
    if 'Day_of_Week' in df.columns:
        df_cleaned['Day_of_Week'] = clean_day_of_week(df['Day_of_Week'])
    
    return df_cleaned

# ---  Execute the cleaning ---
print("\n" + "="*60)
print(" Starting day of week data cleaning...")
print("="*60)

df_cleaned = clean_final_categorical(df_cleaned)

# ---  Final Summary ---
print("="*60)
print(" Day of week cleaning completed successfully! ")
print(f" Final dataset shape: {df_cleaned.shape}")

# Show comprehensive final value counts
if 'Day_of_Week' in df_cleaned.columns:
    print("\n DAY OF WEEK DISTRIBUTION:")
    print("-" * 30)
    
    day_counts = df_cleaned['Day_of_Week'].value_counts()
    total = len(df_cleaned)
    
    # Order days properly for display
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'Unknown']

    
    for day in day_order:
        if day in day_counts:
            count = day_counts[day]
            percentage = (count / total) * 100
            print(f"    {day}: {count} ({percentage:.1f}%)")

print("="*60)


 Starting day of week data cleaning...
 Starting day of week column cleaning...
 Cleaning Day_of_Week column...
    Unique values: 14 → 7
    Final distribution:
      Monday: 35639 (14.3%)
      Tuesday: 35730 (14.3%)
      Wednesday: 35679 (14.3%)
      Thursday: 35986 (14.4%)
      Friday: 35765 (14.3%)
      Saturday: 35554 (14.2%)
      Sunday: 35647 (14.3%)

 Day of week cleaning completed successfully! 
 Final dataset shape: (250000, 28)

📋 DAY OF WEEK DISTRIBUTION:
------------------------------
    Monday: 35639 (14.3%)
    Tuesday: 35730 (14.3%)
    Wednesday: 35679 (14.3%)
    Thursday: 35986 (14.4%)
    Friday: 35765 (14.3%)
    Saturday: 35554 (14.2%)
    Sunday: 35647 (14.3%)


In [328]:
def clean_missing_with_mode(df, columns):
    """
    Impute missing values with mode for specified columns
    """
    print("Starting missing value imputation with mode...")
    print("=" * 50)
    
    df_cleaned = df.copy()
    
    for column in columns:
        if column in df_cleaned.columns:
            missing_count = df_cleaned[column].isna().sum()
            
            if missing_count > 0:
                mode_calc = df_cleaned[column].mode()
                mode_value = mode_calc[0] if not mode_calc.empty else 'Unknown'
                mode_frequency = (df_cleaned[column] == mode_value).sum() if not mode_calc.empty else 0
                
                df_cleaned[column] = df_cleaned[column].fillna(mode_value)
                
                print(f"Column: {column}")
                print(f"  Missing values: {missing_count}")
                print(f"  Mode value: '{mode_value}' (appears {mode_frequency} times)")
                print(f"  Imputed {missing_count} missing values with mode")
                print("-" * 30)
            else:
                print(f"Column: {column} - No missing values found")
                print("-" * 30)
        else:
            print(f"Column '{column}' not found in dataframe")
            print("-" * 30)
    
    return df_cleaned

# Execute the cleaning
print("Cleaning Race_Demographic and ICD-10_Code columns...")
print("=" * 60)

df_cleaned = clean_missing_with_mode(df_cleaned, ['Race_Demographic', 'ICD-10_Code'])

# Show results
print("\nMissing values after cleaning:")
print("=" * 30)
print(f"Race_Demographic: {df_cleaned['Race_Demographic'].isna().sum()}")
print(f"ICD-10_Code: {df_cleaned['ICD-10_Code'].isna().sum()}")

# Show top categories for context
print("\nTop categories after imputation:")
print("=" * 30)
for column in ['Race_Demographic', 'ICD-10_Code']:
    if column in df_cleaned.columns:
        top_category = df_cleaned[column].mode()[0] if not df_cleaned[column].mode().empty else 'N/A'
        print(f"{column}: '{top_category}'")

Cleaning Race_Demographic and ICD-10_Code columns...
Starting missing value imputation with mode...
Column: Race_Demographic
  Missing values: 7399
  Mode value: 'Black' (appears 181781 times)
  Imputed 7399 missing values with mode
------------------------------
Column: ICD-10_Code
  Missing values: 17376
  Mode value: 'R69' (appears 108875 times)
  Imputed 17376 missing values with mode
------------------------------

Missing values after cleaning:
Race_Demographic: 0
ICD-10_Code: 0

Top categories after imputation:
Race_Demographic: 'Black'
ICD-10_Code: 'R69'


In [329]:
import pandas as pd
from dateutil import parser

def clean_messy_datetime(series, reference_date=None):
    """
    Convert a messy datetime series into a proper pandas datetime column.
    Handles multiple formats and missing values.
    
    Parameters:
    - series: pd.Series containing messy datetime strings
    - reference_date: string or pd.Timestamp to fill missing dates or provide default date for times-only values
    
    Returns:
    - pd.Series of cleaned datetime values
    """
    print("Starting datetime cleaning process...")
    print("=" * 50)
    
    def parse_single_datetime(val):
        if pd.isna(val) or str(val).strip() == '':
            return pd.NaT
        val = str(val).strip()
        try:
            # Handle compact numeric formats like 20221229 or 202212290717
            if val.isdigit():
                if len(val) == 12:  # YYYYMMDDHHMM
                    return pd.to_datetime(val, format='%Y%m%d%H%M')
                elif len(val) == 8:  # YYYYMMDD
                    return pd.to_datetime(val, format='%Y%m%d')
            
            # Handle times-only like "10:48" → assume reference_date
            if ':' in val and len(val.split()) == 1 and reference_date is not None:
                return pd.to_datetime(f"{reference_date} {val}")
            
            # Flexible parsing with dateutil
            return parser.parse(val, dayfirst=True, fuzzy=True)
        except Exception:
            return pd.NaT
    
    original_missing = series.isna().sum()
    cleaned_series = series.apply(parse_single_datetime)
    parsed_missing = cleaned_series.isna().sum()
    successfully_parsed = len(series) - parsed_missing
    
    # Fill missing with reference_date if provided
    if reference_date is not None:
        fill_count = cleaned_series.isna().sum()
        cleaned_series = cleaned_series.fillna(pd.Timestamp(reference_date))
        if fill_count > 0:
            print(f"Filled {fill_count} missing values with reference date: {reference_date}")
    
    print(f"Original missing values: {original_missing}")
    print(f"Successfully parsed: {successfully_parsed} values")
    print(f"Final missing values: {cleaned_series.isna().sum()}")
    print(f"Date range: {cleaned_series.min()} to {cleaned_series.max()}")
    print("=" * 50)
    
    return cleaned_series

# Usage example
reference_date = '2023-01-01 12:00:00'
df_cleaned['Arrival_DateTime'] = clean_messy_datetime(df_cleaned['Arrival_Date_Time'], reference_date)

# Drop original column
df_cleaned.drop(columns=['Arrival_Date_Time'], inplace=True)
print("Dropped original Arrival_Date_Time column")

# Extract datetime features
dt_cols = {
    'Arrival_Year': 'year',
    'Arrival_Month': 'month',
    'Arrival_Day': 'day',
    'Arrival_Hour': 'hour',
    'Arrival_Minute': 'minute',
    'Arrival_Weekday': 'weekday',
    'Arrival_DayOfYear': 'dayofyear',
    'Arrival_Quarter': 'quarter'
}

for col_name, attr in dt_cols.items():
    df_cleaned[col_name] = getattr(df_cleaned['Arrival_DateTime'].dt, attr)

print("Created datetime features: Year, Month, Day, Hour, Minute, Weekday, DayOfYear, Quarter")
print(f"Final dataset shape: {df_cleaned.shape}")


Starting datetime cleaning process...
Filled 7555 missing values with reference date: 2023-01-01 12:00:00
Original missing values: 7555
Successfully parsed: 242445 values
Final missing values: 0
Date range: 2022-01-01 00:00:00 to 2023-12-30 23:56:00
Dropped original Arrival_Date_Time column
Created datetime features: Year, Month, Day, Hour, Minute, Weekday, DayOfYear, Quarter
Final dataset shape: (250000, 36)


In [330]:
# Comprehensive Before vs After Cleaning Analysis
print("Generating comprehensive dataset cleaning analysis...")
print("=" * 80)

# Before cleaning summary
before_summary = pd.DataFrame({
    "Missing_Values_Before": df.isna().sum(),
    "Missing_Percent_Before": (df.isna().sum() / len(df)) * 100,
    "Unique_Values_Before": df.nunique(),
    "DataType_Before": df.dtypes
})

# After cleaning summary
after_summary = pd.DataFrame({
    "Missing_Values_After": df_cleaned.isna().sum(),
    "Missing_Percent_After": (df_cleaned.isna().sum() / len(df_cleaned)) * 100,
    "Unique_Values_After": df_cleaned.nunique(),
    "DataType_After": df_cleaned.dtypes
})

# Combine both into one comparison table
comparison = before_summary.join(after_summary, how="outer")

# Calculate improvement metrics
comparison["Missing_Values_Reduction"] = comparison["Missing_Values_Before"] - comparison["Missing_Values_After"]
comparison["Missing_Percent_Reduction"] = comparison["Missing_Percent_Before"] - comparison["Missing_Percent_After"]
comparison["Unique_Values_Change"] = comparison["Unique_Values_After"] - comparison["Unique_Values_Before"]

# Add cleaning status indicator
def get_cleaning_status(row):
    if row["Missing_Values_Reduction"] > 0:
        return "IMPROVED"
    elif row["Missing_Values_Reduction"] == 0 and row["Missing_Values_Before"] == 0:
        return "CLEAN"
    elif row["Missing_Values_Reduction"] == 0:
        return "NO_CHANGE"
    else:
        return "WORSENED"

comparison["Cleaning_Status"] = comparison.apply(get_cleaning_status, axis=1)

# Reorder columns for better readability
column_order = [
    "DataType_Before", "DataType_After",
    "Missing_Values_Before", "Missing_Values_After", "Missing_Values_Reduction",
    "Missing_Percent_Before", "Missing_Percent_After", "Missing_Percent_Reduction",
    "Unique_Values_Before", "Unique_Values_After", "Unique_Values_Change",
    "Cleaning_Status"
]
comparison = comparison[column_order]



# Generate summary statistics
total_missing_reduced = comparison["Missing_Values_Reduction"].sum()
total_columns_improved = (comparison["Cleaning_Status"] == "IMPROVED").sum()
total_columns_clean = (comparison["Cleaning_Status"] == "CLEAN").sum()

# Print comprehensive report
print("\n" + "=" * 80)
print("DATASET CLEANING SUMMARY REPORT")
print("=" * 80)

print(f"\nOverall Statistics:")
print(f"- Total missing values reduced: {total_missing_reduced:,}")
print(f"- Columns improved: {total_columns_improved} / {len(comparison)}")
print(f"- Columns already clean: {total_columns_clean} / {len(comparison)}")
print(f"- Dataset shape before: {df.shape}")
print(f"- Dataset shape after: {df_cleaned.shape}")

print(f"\nTop 5 Most Improved Columns (by missing values reduced):")
most_improved = comparison.nlargest(5, "Missing_Values_Reduction")
for idx, row in most_improved.iterrows():
    if row["Missing_Values_Reduction"] > 0:
        print(f"  - {idx}: Reduced {row['Missing_Values_Reduction']:,} missing values "
              f"({row['Missing_Percent_Reduction']:.1f}%)")

print(f"\nColumns with Remaining Missing Values:")
remaining_missing = comparison[comparison["Missing_Values_After"] > 0]
if len(remaining_missing) > 0:
    for idx, row in remaining_missing.iterrows():
        print(f"  - {idx}: {row['Missing_Values_After']:,} missing values "
              f"({row['Missing_Percent_After']:.1f}%)")
else:
    print("  - All missing values have been addressed!")

print(f"\nData Type Changes:")
type_changes = comparison[comparison["DataType_Before"] != comparison["DataType_After"]]
if len(type_changes) > 0:
    for idx, row in type_changes.iterrows():
        print(f"  - {idx}: {row['DataType_Before']} → {row['DataType_After']}")
else:
    print("  - No data type changes")

print(f"\nSample of Comparison Table (first 10 rows):")
print("=" * 100)
print(comparison.head(10).to_string(float_format=lambda x: f"{x:.1f}" if isinstance(x, float) else str(x)))


print("=" * 80)

# Additional: Memory usage comparison
memory_before = df.memory_usage(deep=True).sum() / 1024**2  # MB
memory_after = df_cleaned.memory_usage(deep=True).sum() / 1024**2  # MB
memory_change = memory_after - memory_before

print(f"\nMemory Usage Comparison:")
print(f"- Before cleaning: {memory_before:.2f} MB")
print(f"- After cleaning: {memory_after:.2f} MB")
print(f"- Memory change: {memory_change:+.2f} MB ({memory_change/memory_before*100:+.1f}%)")

print("=" * 80)
print(" Comprehensive dataset cleaning analysis completed!")

Generating comprehensive dataset cleaning analysis...

DATASET CLEANING SUMMARY REPORT

Overall Statistics:
- Total missing values reduced: 196,546.0
- Columns improved: 9 / 39
- Columns already clean: 17 / 39
- Dataset shape before: (250000, 29)
- Dataset shape after: (250000, 36)

Top 5 Most Improved Columns (by missing values reduced):
  - Heart_Rate: Reduced 37,645.0 missing values (15.1%)
  - Oxygen_Saturation: Reduced 37,645.0 missing values (15.1%)
  - Respiratory_Rate: Reduced 37,645.0 missing values (15.1%)
  - SATS_Score: Reduced 37,645.0 missing values (15.1%)
  - ICD-10_Code: Reduced 17,376.0 missing values (7.0%)

Columns with Remaining Missing Values:
  - All missing values have been addressed!

Data Type Changes:
  - Age_Category: nan → category
  - Arrival_DateTime: nan → datetime64[ns]
  - Arrival_Date_Time: object → nan
  - Arrival_Day: nan → int32
  - Arrival_DayOfYear: nan → int32
  - Arrival_Hour: nan → int32
  - Arrival_Minute: nan → int32
  - Arrival_Month: nan →

In [332]:
import os
import shutil

# Save cleaned data in the three requested formats to the correct directory
print("Saving cleaned data to '06_Data/Cleaned and Processed Data' folder...")
print("=" * 60)

# Define the correct target directory path
target_directory = "../06_Data/Cleaned and Processed Data"

# Create the directory if it doesn't exist
os.makedirs(target_directory, exist_ok=True)
print(f" Target directory ensured: {target_directory}")

# Create timestamp for file names
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
base_filename = f"cleaned_emergency_data_{timestamp}"

# 1. Save as CSV (most compatible)
csv_filename = f"{base_filename}.csv"
csv_filepath = os.path.join(target_directory, csv_filename)
df_cleaned.to_csv(csv_filepath, index=False, encoding='utf-8')
csv_size = os.path.getsize(csv_filepath) / 1024**2  # Get actual file size in MB
print(f" CSV file saved: {csv_filepath}")
print(f"   - Size: {csv_size:.2f} MB")
print(f"   - Records: {len(df_cleaned):,}")
print(f"   - Columns: {len(df_cleaned.columns)}")

# 2. Save as Parquet (efficient for large datasets)
parquet_filename = f"{base_filename}.parquet"
parquet_filepath = os.path.join(target_directory, parquet_filename)
df_cleaned.to_parquet(parquet_filepath, index=False, engine='pyarrow', compression='snappy')
parquet_size = os.path.getsize(parquet_filepath) / 1024**2
print(f" Parquet file saved: {parquet_filepath}")
print(f"   - Size: {parquet_size:.2f} MB ({parquet_size/csv_size*100:.1f}% of CSV size)")
print(f"   - Compression efficiency: {csv_size/parquet_size:.1f}x better than CSV")

# 3. Save as Excel (user-friendly)
excel_filename = f"{base_filename}.xlsx"
excel_filepath = os.path.join(target_directory, excel_filename)

# Check if dataset is within Excel limits
if len(df_cleaned) <= 1048576:  # Excel row limit
    df_cleaned.to_excel(excel_filepath, index=False, engine='openpyxl')
    excel_size = os.path.getsize(excel_filepath) / 1024**2
    print(f" Excel file saved: {excel_filepath}")
    print(f"   - Size: {excel_size:.2f} MB")
else:
    print(f"  Excel file skipped: Dataset has {len(df_cleaned):,} rows, exceeds Excel's 1,048,576 row limit")
    # Create a sample Excel file with the first 100k rows instead
    sample_excel_filename = f"{base_filename}_sample_100k.xlsx"
    sample_excel_filepath = os.path.join(target_directory, sample_excel_filename)
    sample_size = min(100000, len(df_cleaned))
    df_cleaned.head(sample_size).to_excel(sample_excel_filepath, index=False, engine='openpyxl')
    sample_excel_size = os.path.getsize(sample_excel_filepath) / 1024**2
    print(f" Sample Excel file saved: {sample_excel_filepath}")
    print(f"   - Size: {sample_excel_size:.2f} MB")
    print(f"   - Contains: {sample_size:,} records (sample)")

# Create a metadata file in the same directory
metadata_filename = f"{base_filename}_metadata.txt"
metadata_filepath = os.path.join(target_directory, metadata_filename)

with open(metadata_filepath, 'w') as f:
    f.write("UBUNTUCARE - SMARTER WAIT TIMES & OUTCOMES\n")
    f.write("=" * 50 + "\n")
    f.write("CLEANED EMERGENCY DEPARTMENT DATASET\n")
    f.write("=" * 50 + "\n\n")
    f.write(f"Project: Emergency Department Optimization\n")
    f.write(f"Cleaning completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Total records: {len(df_cleaned):,}\n")
    f.write(f"Total columns: {len(df_cleaned.columns)}\n")
    f.write(f"Total missing values: {df_cleaned.isna().sum().sum():,}\n\n")
    
    f.write("FILE INFORMATION:\n")
    f.write("-" * 20 + "\n")
    f.write(f"CSV file: {csv_filename} ({csv_size:.1f} MB)\n")
    f.write(f"Parquet file: {parquet_filename} ({parquet_size:.1f} MB)\n")
    if len(df_cleaned) <= 1048576:
        f.write(f"Excel file: {excel_filename} ({excel_size:.1f} MB)\n")
    else:
        f.write(f"Excel sample: {sample_excel_filename}\n")
    f.write(f"Location: {target_directory}\n\n")
    
    f.write("DATA CLEANING SUMMARY:\n")
    f.write("-" * 25 + "\n")
    f.write("Performed operations:\n")
    f.write("- Age standardization and categorization\n")
    f.write("- Waiting time outlier handling\n")
    f.write("- Vital signs cleaning and imputation\n")
    f.write("- Categorical data standardization\n")
    f.write("- Gender and triage category cleaning\n")
    f.write("- Datetime parsing and feature extraction\n")
    f.write("- Missing value imputation\n\n")
    
    f.write("RECOMMENDED USAGE:\n")
    f.write("-" * 20 + "\n")
    f.write("Analysis: Use Parquet format (fastest)\n")
    f.write("Sharing: Use CSV format (most compatible)\n")
    f.write("Reporting: Use Excel format (user-friendly)\n")

print(f" Metadata file saved: {metadata_filepath}")

# Verify the files were saved in the correct location
print(f"\n Verifying files in '{target_directory}':")
print("-" * 50)
files_in_dir = os.listdir(target_directory)
cleaned_files = [f for f in files_in_dir if f.startswith(base_filename)]

if cleaned_files:
    for file in sorted(cleaned_files):
        file_path = os.path.join(target_directory, file)
        file_size = os.path.getsize(file_path) / 1024**2
        print(f" {file} ({file_size:.2f} MB)")
else:
    print(" No cleaned files found in target directory!")

# Double-check by showing absolute path
absolute_path = os.path.abspath(target_directory)
print(f"\n Absolute path: {absolute_path}")

print("\n" + "=" * 60)
print("SAVING COMPLETE - SUMMARY:")
print("=" * 60)
print(f" Target directory: {target_directory}")
print(f" Files created: {len(cleaned_files)}")
print(f" Total size: {sum(os.path.getsize(os.path.join(target_directory, f)) for f in cleaned_files) / 1024**2:.2f} MB")
print(f" Dataset shape: {df_cleaned.shape}")

print("\n" + "=" * 60)
print(" All files successfully saved to '06_Data/Cleaned and Processed Data' folder!")
print(" Ready for next steps in the UBUNTUCARE pipeline!")
print("=" * 60)

Saving cleaned data to '06_Data/Cleaned and Processed Data' folder...
 Target directory ensured: ../06_Data/Cleaned and Processed Data
 CSV file saved: ../06_Data/Cleaned and Processed Data\cleaned_emergency_data_20250916_174258.csv
   - Size: 66.23 MB
   - Records: 250,000
   - Columns: 36
 Parquet file saved: ../06_Data/Cleaned and Processed Data\cleaned_emergency_data_20250916_174258.parquet
   - Size: 8.86 MB (13.4% of CSV size)
   - Compression efficiency: 7.5x better than CSV
 Excel file saved: ../06_Data/Cleaned and Processed Data\cleaned_emergency_data_20250916_174258.xlsx
   - Size: 49.44 MB
 Metadata file saved: ../06_Data/Cleaned and Processed Data\cleaned_emergency_data_20250916_174258_metadata.txt

 Verifying files in '../06_Data/Cleaned and Processed Data':
--------------------------------------------------
 cleaned_emergency_data_20250916_174258.csv (66.23 MB)
 cleaned_emergency_data_20250916_174258.parquet (8.86 MB)
 cleaned_emergency_data_20250916_174258.xlsx (49.44 MB